#**THE ALGO TRADING CAPSTONE**

---

##0.REFERENCE

https://claude.ai/share/b697309d-44d8-43f4-8bbc-d768b505cfab

##1.THE COMPLETE PIPELINE

This capstone notebook is the culmination of the entire book because it does not merely “run a strategy.” It demonstrates how an algorithmic trading system can be built as a **governance-native product**: something you can test, audit, reproduce, promote through gates, and operate in production with monitoring and cross-functional handoffs. In other words, this notebook is not a research script. It is a complete **system lifecycle demonstration**: from synthetic data scaffolding, to feature and signal design, to portfolio and risk, to execution costs, to backtesting, to validation, to governance approvals, to agent-style memos, and finally to a **production-ready artifact pack**.

A key theme throughout is **separation of concerns**. Each layer does one job, writes explicit artifacts, and passes outputs forward. The goal is that you can review each step independently, replace a module without rewriting the whole system, and prove what happened in a run by inspecting saved manifests and hashes. This is how you move from experimentation to something that survives real-world scrutiny.

**How to Read This Section**

The structure mirrors the notebook: each subsection corresponds to one cell and explains (1) what that cell does, (2) why it exists in a production-grade workflow, and (3) what artifacts it produces for audit, review, and handoff. The final sections then reassemble the full story: **research → validation → governance → promotion → production operations**.

**Cell 1 — Run Identity, Determinism, and the Audit Directory**

The first cell establishes a foundational mindset: **this is a system run, not a casual analysis**. It sets a deterministic seed and uses it consistently so that results can be reproduced. That is the earliest point at which governance begins, because reproducibility is the minimum requirement for any claim that can be audited.

Next, the notebook creates a run identifier. In real production you often include timestamps, but here it is derived from the seed to keep teaching runs deterministic and easy to compare across students, machines, and environments.

Most importantly, the notebook creates a directory structure that becomes the backbone of the entire run. Each directory represents a domain responsibility: data, features, signals, portfolio, risk, execution, validation, monitoring, governance, reports, and agents. This is one of the strongest “capstone” statements in the notebook: **the filesystem is not storage, it is the audit trail**. By the end, your results are not just numbers on a screen; they are a complete run record.

**Cell 2 — The Single Source of Truth Configuration**

This cell formalizes the system’s operating contract as one configuration object. Every module downstream reads from it so that assumptions are not scattered across the notebook or hidden inside functions.

The configuration covers the entire lifecycle: synthetic data parameters, time semantics, feature lookbacks and lags, signal toggles, regime detection policy, portfolio constraints, risk overlays, execution costs, backtest benchmarking, validation requirements, and governance requirements. This is how the notebook enforces consistency.

The cell then saves the configuration to disk and computes a hash. That hash becomes a run “fingerprint.” The practical message is simple and powerful: **if the config changes, the run changed**. This will matter later when reproducibility is tested, and it matters in the real world when version tracking is required.

**Cell 3 — Governance Utilities and the Artifact Registry**

Here the notebook builds its internal infrastructure: hashing utilities, safe writing utilities, and time-discipline assertions. This is a separation-of-concerns milestone: you do not want strategy logic mixed with operational plumbing.

The hashing and safe I/O functions exist so that outputs are written deterministically and can be fingerprinted. The monotonic-time and no-lookahead checks exist so the system can prove it is not using future information.

The Artifact Registry introduced here is one of the defining capstone ideas: every important output is recorded with name, path, hash, schema version, dependencies, and timestamp. This transforms “a notebook run” into **a traceable lineage graph**. Later, governance and agent memos can point to artifacts by path and hash, instead of relying on informal descriptions.

**Cell 4 — Synthetic Data as a Governed Research Environment**

This cell generates synthetic market data, but the emphasis is not realism for its own sake. The emphasis is building a controlled environment where system architecture can be tested safely and deterministically.

The generator creates a constant universe of tickers, produces price paths with regime-switching volatility, generates volumes suitable for transaction cost modeling, simulates missing data to test robustness, and creates corporate action flags to force the pipeline to handle adjustments.

Crucially, it does not just create arrays; it creates governed artifacts: a universe manifest, raw prices, raw volumes, regime state, corporate actions, a data quality report, and a data fingerprint. Then it registers these artifacts. This is the capstone posture: **data is a first-class audited object**.

**Cell 5 — Data Normalization, Adjustments, and Alignment Proof**

This cell converts raw inputs into standardized trading inputs while preserving causality. It demonstrates three core engineering responsibilities.

First, it applies a corporate action adjustment policy. The policy is simplified, but the architectural point is that the policy itself is saved as an artifact. Reviewers can see exactly what rule was used.

Second, it handles missing data using forward filling, which is causal because it only uses past information. It also treats early missing values with a cautious backfill approach so the series can start.

Third, it computes returns from adjusted prices and replaces problematic values with safe defaults. That is a fail-closed choice: the system should behave conservatively in the face of data imperfections.

Finally, it writes an alignment proof artifact documenting assumptions like constant universe and applied adjustments. This is a subtle but important governance lesson: **you do not only want data; you want evidence about the data**.

**Cell 6 — Feature Engineering with Causality Gates**

This cell builds features, but the heart of it is time discipline. Features are computed with explicit lookbacks and then lagged so that a feature at a given time reflects only information available before decisions are made.

The notebook builds momentum, volatility, and mean reversion style features, then optionally standardizes momentum cross-sectionally to produce a relative signal base. It replaces missing feature values with safe defaults to avoid fragile behavior.

It also produces leakage tests and a feature manifest as artifacts. Whether the tests are minimal or extensive, the principle is the same: **feature engineering must come with causality evidence**. This is part of what makes the capstone a culmination: features are no longer “research creativity,” they are “auditable inputs.”

**Cell 7 — Baseline Signals as Modular Strategy Components**

This cell draws a clear boundary: features are measurements; signals are decisions. It builds modular baseline signals: trend, mean reversion, and inverse volatility inputs for scaling.

The decision to implement these signals as separate functions is not just programming style. It is architectural separation. Each signal is a component that can be swapped, tuned, enabled or disabled, and tested independently.

The cell also produces diagnostics such as distribution summaries and a turnover proxy. That is a practical production lesson: **a signal has to be tradable, not just predictive**. Diagnostics anticipate execution and costs before you even build the execution engine.

**Cell 8 — Regime Detection and Routing with Strict Timing**

This cell adds adaptivity without introducing a black box. It computes a volatility-based regime state using only past data. That prevents a classic research pitfall: regime labels that accidentally “see” the future.

Then it defines a routing policy: in low volatility, lean more into trend; in high volatility, lean more into mean reversion. The regime state is not the alpha itself; it is a context label that changes how signals are blended.

The routing policy is written as a separate artifact. That makes governance easier: you can review and approve the policy as a document, not as scattered code logic.

**Cell 9 — Signal Combination and Contribution Tracking**

This cell assembles the baseline signals into a final combined signal. It applies regime-based weights and then applies a volatility-based scaling so that the same signal does not imply the same aggressiveness in every environment.

The key architectural point is that the combination layer is its own responsibility. It is not portfolio construction. It is not execution. It is a signal assembly step.

It also tracks contributions in a summary form. Even simplified contribution tracking helps answer a common question from review committees: “Which component is driving decisions most of the time?” This is interpretability through design rather than after-the-fact explanations.

**Cell 10 — Portfolio Construction Under Explicit Constraints**

This cell converts signals into portfolio weights. In the book arc, this is where ideas become allocations.

It uses a transparent heuristic approach to produce weights, then enforces constraints: per-asset caps, a leverage bound, and dollar neutrality. It also rescale-checks after neutrality adjustment, which is important because constraints can interact.

The solver trace is an operational goldmine. It records how often constraints bind, which is exactly what production teams and risk committees want to know. If you are always clipping or rescaling, your signal might be too aggressive or your constraints might be too tight. Either way, the trace points to the real tension.

Artifacts produced here prove compliance: target weights, constraints bounds, and a manifest describing the method and limits.

**Cell 11 — Risk Overlays as a Separate Control Layer**

This cell is where the notebook demonstrates a mature separation: portfolio construction expresses alpha under constraints, while risk overlays impose survival rules and adaptive exposure.

It applies volatility targeting to maintain consistent risk, drawdown control to reduce exposure when losses accumulate, and a circuit breaker to stop trading after extreme single-period losses.

It logs events from each overlay. This is essential for explaining behavior. In production, the question is rarely “what was the return?” The question is often “why did the system reduce exposure here?” Event logs are the difference between a defensible system and a mystery machine.

Artifacts produced include final weights, a risk manifest, and risk event logs.

**Cell 12 — Execution Simulation and Transaction Cost Reality**

This cell forces the system to confront real trading frictions.

It enforces execution delay so that decisions are separated from fills. It models spread and fees as proportional costs, and models impact as a function of trade size relative to volume. It computes turnover and compares it to a daily cap derived from an annual turnover constraint.

This is one of the strongest “culmination” moments because it rewires the mindset: performance that ignores execution is not performance; it is a hypothetical. This cell ensures the backtest later is net of costs and timing-correct.

Even the note about turnover capping being applied post-hoc is pedagogically valuable: it distinguishes teaching simplicity from production behavior while keeping the architecture aligned with real systems.

Artifacts produced here are exactly what an execution review requires: orders, fills, costs, turnover, and an execution manifest.

**Cell 13 — Backtesting with Correct Timing, Benchmarking, and Evidence Artifacts**

This cell evaluates the system end-to-end. It computes gross returns and net returns with costs included. It also computes a benchmark and equity curves for comparison.

It produces performance metrics, cost impact summaries, and a failure-mode list that highlights problematic periods. It saves a report and visualizations, turning backtest results into durable evidence.

A subtle but critical point is that timing rules remain consistent: holdings are applied to subsequent returns, and costs are applied at execution times. The backtest is not just a computation; it is a policy-compliant evaluation.

Artifacts produced here become “review ready”: equity curves, backtest report, attribution summary, and failure modes.

**Cell 14 — Minimal End-to-End Test Suite**

This cell formalizes the idea that a backtest is insufficient without safety checks.

It runs tests for monotonic time, no-lookahead boundaries, reproducibility via hashes, cost sanity, turnover cap enforcement, and fail-closed behavior for cleaned features.

The output is a test results artifact that can be referenced in governance gates. This is how you replace informal confidence with documented evidence: **tests are a first-class artifact**.

**Cell 15 — Governance Acceptance Criteria and the Promotion Gate**

Here the notebook steps into organizational reality. It defines acceptance criteria that combine correctness and plausibility. It requires causality and reproducibility, checks cost sanity, and uses a simple performance plausibility threshold.

It produces an acceptance draft with overall status and supporting hashes, then produces a governance pack with approvals placeholders and audit trail pointers. It also produces a monitoring pack defining the metrics that must be tracked and the thresholds that should trigger alerts.

This is the moment when “research” becomes “candidate for production.” It creates the documents that allow formal promotion.

**Cell 16 — Agentic Handoff: Cross-Functional Memos and Bundling**

This cell demonstrates how system results move through people and roles, not just code modules. It simulates three specialized agents that read artifacts and produce memos:

- ResearchAgent focuses on performance narrative, costs, drawdowns, and improvement suggestions.
- RiskAgent focuses on limits, drawdown behavior, and control activations.
- OpsAgent focuses on data quality and execution sustainability.

Each memo lists its inputs and includes hashes so the memo is traceable to a specific run and artifact set. Then a handoff bundle aggregates memos and points to key governance artifacts.

This is the capstone’s “organization layer”: production is the coordination of research, risk, and operations with shared evidence.

**Cell 17 — Final Summary, Artifact Index, and Production-Style Packaging**

This cell packages everything into a run-level deliverable: it saves the artifact registry, writes an artifact index, generates a run summary report, and produces monitoring-relevant plots like drawdown and turnover.

It reads like a deployment attachment. It shows exactly where key artifacts live and gives reviewers a map of the run without requiring them to rerun the notebook.

This is the final transformation: the notebook becomes a governed deliverable, not an interactive experiment.

**Cell 18 — Optional Real Data Adapter, Cleanly Isolated**

The notebook ends with an optional real-data adapter that is disabled by default. The design decision is intentional: synthetic-first keeps runs deterministic and teaching-friendly. Real data introduces external variability and dependency risk.

When enabled, the adapter fetches real prices and volumes and stores them separately so the synthetic run remains intact. It demonstrates how the pipeline can be re-used with real inputs without changing core architecture.

This final cell is the bridge: **research pipeline first, real-world integration second**, without breaking governance discipline.

**The End-to-End Lifecycle in One Narrative**

What the capstone truly demonstrates is a disciplined lifecycle:

- Research foundation: deterministic run identity, configuration, governed synthetic data.
- Engineering pipeline: normalization, features, modular signals, regime routing, combination.
- Allocation and safety: portfolio constraints, risk overlays, kill switches.
- Trading realism: execution delay, cost modeling, turnover caps.
- Evidence and review: backtest reports, failure modes, visual artifacts.
- Validation: reproducibility checks, no-lookahead checks, sanity checks, fail-closed behavior.
- Promotion: acceptance criteria, governance pack, monitoring contract.
- Organization and operations: agentic memos, handoff bundle, packaged artifact index.
- Deployment readiness: a run folder that functions as an audit trail and a review packet.

**Where Separation of Concerns Is Most Visible**

This notebook’s architecture is strongest where boundaries are crisp:

- Configuration vs implementation: one contract, many modules.
- Utilities vs domain logic: hashing and IO do not pollute strategy code.
- Data vs features vs signals: each layer outputs its own governed artifacts.
- Signal assembly vs portfolio construction: scoring is separate from allocation.
- Portfolio construction vs risk overlays: alpha expression is separate from survival controls.
- Execution vs evaluation: trading frictions are explicit, not assumed away.
- Evaluation vs validation: evidence is separate from safety certification.
- Governance vs analytics: promotion is documented as artifacts and criteria.
- System vs organization: memos and bundles demonstrate cross-functional review flow.

**Closing Perspective**

Chapter 25 does not claim that this exact strategy is the edge. The deeper message is that an edge is not credible unless the system is **reviewable, reproducible, governable, and operable**. This capstone notebook shows how to build that discipline end-to-end: a modular research pipeline that generates artifacts, passes tests, satisfies acceptance criteria, produces monitoring contracts, and supports cross-functional handoffs—exactly the path from research to promotion to production.


##2.CODE AND IMPLEMENTATION

In [ ]:
# Cell 1 — Title, deterministic seed, environment, and run ID

"""
FOUNDATIONS OF MODERN ALGORITHMIC TRADING
Chapter 25: Capstone — Building an Auditable, Governance-Native AI Trading System

This notebook implements a complete end-to-end trading system with:
- Strict time awareness and causality
- Modular architecture (data → features → signals → portfolio → risk → execution → backtest)
- Governance-native artifacts (manifests, hashes, lineage, audit trails)
- Deterministic reproducibility
- No pandas - pure NumPy and Python standard library

Target audience: MBA/MFin/practitioners seeking production-grade system design patterns
"""

import numpy as np
import random
import json
import hashlib
import os
from pathlib import Path
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt

# Set global seed for deterministic runs
SEED = 42
np.random.seed(SEED)
random.seed(SEED)

# Create deterministic RUN_ID from seed for reproducibility
# In production, you'd use timestamp, but for teaching we make it seed-based
random.seed(SEED)
run_suffix = ''.join(random.choices('0123456789abcdef', k=8))
RUN_ID = f"run_{SEED}_{run_suffix}"

# Create directory structure for artifacts
BASE_DIR = Path(f"runs/{RUN_ID}")
DIRS = {
    'manifest': BASE_DIR / 'manifest',
    'data': BASE_DIR / 'data',
    'features': BASE_DIR / 'features',
    'signals': BASE_DIR / 'signals',
    'portfolio': BASE_DIR / 'portfolio',
    'risk': BASE_DIR / 'risk',
    'execution': BASE_DIR / 'execution',
    'validation': BASE_DIR / 'validation',
    'monitoring': BASE_DIR / 'monitoring',
    'governance': BASE_DIR / 'governance',
    'reports': BASE_DIR / 'reports',
    'agents': BASE_DIR / 'agents',
}

for dir_path in DIRS.values():
    dir_path.mkdir(parents=True, exist_ok=True)

print(f"=== RUN METADATA ===")
print(f"RUN_ID: {RUN_ID}")
print(f"SEED: {SEED}")
print(f"Base Directory: {BASE_DIR}")
print(f"Timestamp: {datetime.now().isoformat()}")
print(f"\n=== DIRECTORY TREE ===")
for name, path in DIRS.items():
    print(f"{name:15s}: {path}")
print()

# Cell 2 — Global conventions and configuration schema (single source of truth)

"""
Configuration Schema: Single Source of Truth

This CONFIG dictionary defines all parameters for the trading system.
Every module reads from this config to ensure consistency.
Changes here propagate throughout the system deterministically.
"""

CONFIG = {
    # Data generation
    'data': {
        'num_assets': 10,  # Universe size
        'num_bars': 1000,  # Time series length
        'initial_price': 100.0,
        'drift': 0.0001,  # Per-bar drift (annualized would multiply by sqrt(252))
        'base_volatility': 0.015,  # Per-bar volatility
        'regime_vol_low': 0.010,
        'regime_vol_high': 0.025,
        'regime_transition_prob': 0.02,  # Probability of regime switch per bar
        'missingness_rate': 0.001,  # Probability of missing data point
    },

    # Universe and time semantics
    'universe': {
        'ticker_prefix': 'A',
        'constant_universe': True,  # No additions/removals during backtest
    },

    'time_semantics': {
        'bar_interval': 'daily',  # Label only (actual bars are just indices 0..T-1)
        'decision_time_rule': 'bar_close',  # Signals computed at bar close
        'execution_delay_bars': 1,  # Trade executes 1 bar after decision
        'timezone_string': 'UTC',  # Label only
        'trading_calendar_type': 'synthetic_continuous',  # No holidays in synthetic data
    },

    # Feature engineering
    'feature': {
        'momentum_lookback': 20,
        'volatility_lookback': 20,
        'mean_reversion_lookback': 5,
        'cross_sectional_zscore': True,
        'causality_lag': 1,  # All features lagged by this many bars
    },

    # Signal generation
    'signal': {
        'trend_enabled': True,
        'mean_reversion_enabled': True,
        'volatility_targeting_enabled': True,
    },

    # Regime detection
    'regime': {
        'enabled': True,
        'detection_method': 'rolling_volatility',
        'volatility_threshold': 0.018,  # Above this = high-vol regime
        'lookback': 20,
    },

    # Portfolio construction
    'portfolio': {
        'risk_model': 'diagonal',  # Diagonal covariance (no correlations)
        'max_leverage': 1.0,  # L1 norm of weights <= 1.0
        'max_weight_per_asset': 0.3,
        'neutrality_constraint': 'dollar_neutral',  # sum(weights) = 0
        'optimization_method': 'heuristic_mean_variance',
    },

    # Risk management
    'risk': {
        'volatility_target': 0.12,  # Annualized target (daily would be ~0.12/sqrt(252))
        'volatility_target_lookback': 60,
        'drawdown_threshold': 0.15,  # Max 15% drawdown before reducing exposure
        'drawdown_response': 'scale_down',  # 'scale_down' or 'go_flat'
        'drawdown_scale_factor': 0.5,
        'circuit_breaker_enabled': True,
        'circuit_breaker_loss_threshold': 0.05,  # Single-bar loss > 5% triggers circuit breaker
    },

    # Execution and costs
    'execution': {
        'spread_bps': 5,  # 5 basis points spread
        'fee_bps': 1,  # 1 basis point commission
        'impact_coefficient': 0.1,  # Market impact = coeff * (trade/volume)
        'turnover_cap_annual': 10.0,  # Max 10x annual turnover (daily cap = 10/252)
    },

    # Backtest
    'backtest': {
        'benchmark': 'equal_weight',
        'include_costs': True,
    },

    # Validation
    'validation': {
        'causality_checks': True,
        'reproducibility_checks': True,
        'cost_sanity_checks': True,
        'fail_closed_test': True,
    },

    # Governance
    'governance': {
        'require_acceptance_criteria': True,
        'require_artifact_registry': True,
        'require_audit_trail': True,
    },
}

# Save config
config_path = DIRS['manifest'] / 'config.json'
with open(config_path, 'w') as f:
    json.dump(CONFIG, f, indent=2, sort_keys=True)

# Compute config hash for version tracking
config_json = json.dumps(CONFIG, sort_keys=True)
config_hash = hashlib.sha256(config_json.encode()).hexdigest()

print("=== CONFIGURATION ===")
print(f"Config saved to: {config_path}")
print(f"Config hash: {config_hash[:16]}...")
print(f"\nKey parameters:")
print(f"  Assets: {CONFIG['data']['num_assets']}")
print(f"  Bars: {CONFIG['data']['num_bars']}")
print(f"  Execution delay: {CONFIG['time_semantics']['execution_delay_bars']}")
print(f"  Max leverage: {CONFIG['portfolio']['max_leverage']}")
print(f"  Regime enabled: {CONFIG['regime']['enabled']}")
print()

# Cell 3 — Utility functions (hashing, IO, asserts)

"""
Utility Functions: Infrastructure for Governance and Safety

These functions provide:
- Cryptographic hashing for artifact versioning
- Safe I/O with automatic directory creation
- Causality and time-order assertions
- Artifact registry for lineage tracking
"""

def sha256_bytes(data: bytes) -> str:
    """Compute SHA-256 hash of bytes."""
    return hashlib.sha256(data).hexdigest()

def sha256_file(filepath: Path) -> str:
    """Compute SHA-256 hash of file contents."""
    with open(filepath, 'rb') as f:
        return sha256_bytes(f.read())

def sha256_json(obj) -> str:
    """Compute SHA-256 hash of JSON-serializable object."""
    json_str = json.dumps(obj, sort_keys=True)
    return sha256_bytes(json_str.encode())

def safe_write_json(filepath: Path, obj, indent=2):
    """Write JSON with deterministic key ordering."""
    filepath.parent.mkdir(parents=True, exist_ok=True)
    with open(filepath, 'w') as f:
        json.dump(obj, f, indent=indent, sort_keys=True)

def safe_write_npy(filepath: Path, array: np.ndarray):
    """Write NumPy array to .npy file."""
    filepath.parent.mkdir(parents=True, exist_ok=True)
    np.save(filepath, array)

def safe_write_txt(filepath: Path, text: str):
    """Write text file."""
    filepath.parent.mkdir(parents=True, exist_ok=True)
    with open(filepath, 'w') as f:
        f.write(text)

def assert_monotonic_time(t: np.ndarray, name: str = "time"):
    """Assert that time array is strictly increasing."""
    assert len(t) > 0, f"{name}: empty array"
    if len(t) > 1:
        diffs = np.diff(t)
        assert np.all(diffs > 0), f"{name}: not strictly increasing at indices {np.where(diffs <= 0)[0]}"

def assert_no_lookahead(max_input_time: int, decision_time: int, context: str = ""):
    """
    Assert causality: inputs must come from strictly before decision time.

    max_input_time: latest time index used in input data
    decision_time: time index at which decision is made
    context: description for error messages
    """
    assert max_input_time < decision_time, \
        f"LOOKAHEAD VIOLATION {context}: max_input_time={max_input_time} >= decision_time={decision_time}"

# Artifact Registry: Track all produced artifacts with lineage
ARTIFACT_REGISTRY = []

def add_artifact(name: str, path: Path, hash_value: str, schema_version: str, depends_on: list = None):
    """
    Register an artifact in the global registry.

    Args:
        name: Human-readable artifact name
        path: File path (relative to BASE_DIR preferred)
        hash_value: SHA-256 hash of artifact
        schema_version: Version string for artifact schema
        depends_on: List of artifact names this depends on
    """
    artifact = {
        'name': name,
        'path': str(path.relative_to(BASE_DIR)) if path.is_relative_to(BASE_DIR) else str(path),
        'hash': hash_value,
        'schema_version': schema_version,
        'depends_on': depends_on or [],
        'timestamp': datetime.now().isoformat(),
    }
    ARTIFACT_REGISTRY.append(artifact)

def save_artifact_registry():
    """Save artifact registry to manifest directory."""
    registry_path = DIRS['manifest'] / 'artifact_registry.json'
    safe_write_json(registry_path, ARTIFACT_REGISTRY)
    print(f"Artifact registry saved: {len(ARTIFACT_REGISTRY)} artifacts")

print("=== UTILITIES LOADED ===")
print("Available functions:")
print("  - Hashing: sha256_bytes, sha256_file, sha256_json")
print("  - I/O: safe_write_json, safe_write_npy, safe_write_txt")
print("  - Assertions: assert_monotonic_time, assert_no_lookahead")
print("  - Registry: add_artifact, save_artifact_registry")
print()

# Cell 4 — Synthetic data generator (universe + prices + volumes + corporate actions flags)

"""
Synthetic Data Generation

Generates realistic multi-asset price data with:
- Regime-switching volatility (Markov chain)
- Corporate actions (splits, dividends)
- Missing data simulation
- Strict time ordering (no lookahead in regime generation)

Key principle: Regime is generated forward in time using only past information.
"""

# Extract config
N = CONFIG['data']['num_assets']
T = CONFIG['data']['num_bars']
P0 = CONFIG['data']['initial_price']
drift = CONFIG['data']['drift']
vol_low = CONFIG['data']['regime_vol_low']
vol_high = CONFIG['data']['regime_vol_high']
regime_transition_prob = CONFIG['data']['regime_transition_prob']
missingness_rate = CONFIG['data']['missingness_rate']

# Time index: 0, 1, 2, ..., T-1
time_index = np.arange(T)
assert_monotonic_time(time_index, "time_index")

# Generate regime state (0 = low vol, 1 = high vol)
# This is a forward simulation - no lookahead
regime_state = np.zeros(T, dtype=int)
regime_state[0] = 0  # Start in low-vol regime

for t in range(1, T):
    if np.random.random() < regime_transition_prob:
        regime_state[t] = 1 - regime_state[t-1]  # Switch regime
    else:
        regime_state[t] = regime_state[t-1]  # Stay in same regime

# Generate universe: tickers A00, A01, ..., A(N-1)
tickers = [f"{CONFIG['universe']['ticker_prefix']}{i:02d}" for i in range(N)]

universe_manifest = {
    'tickers': tickers,
    'num_assets': N,
    'constant_universe': CONFIG['universe']['constant_universe'],
    'creation_time': datetime.now().isoformat(),
}

# Generate price paths
raw_prices = np.zeros((T, N))
raw_volumes = np.zeros((T, N))

# Initialize prices
raw_prices[0, :] = P0

# Generate returns and prices
for t in range(1, T):
    # Volatility depends on regime at time t
    vol_t = vol_low if regime_state[t] == 0 else vol_high

    # Generate returns: drift + vol * random
    returns = drift + vol_t * np.random.randn(N)

    # Apply returns to prices (geometric)
    raw_prices[t, :] = raw_prices[t-1, :] * np.exp(returns)

# Generate volumes (log-normal around mean based on price)
for i in range(N):
    base_volume = 1000000 * (1 + 0.1 * i)  # Different volume for each asset
    raw_volumes[:, i] = base_volume * np.exp(0.2 * np.random.randn(T))

# Simulate missing data
missing_mask = np.random.random((T, N)) < missingness_rate
raw_prices[missing_mask] = np.nan
raw_volumes[missing_mask] = np.nan

# Corporate actions: simple flags (1 = split/dividend event)
# For simplicity, create sparse events (1% chance per asset per bar)
corp_actions = np.zeros((T, N), dtype=int)
corp_actions[np.random.random((T, N)) < 0.01] = 1
# Don't put corporate actions in first 100 bars to avoid complications
corp_actions[:100, :] = 0

# Save artifacts
safe_write_json(DIRS['data'] / 'universe_manifest.json', universe_manifest)
safe_write_npy(DIRS['data'] / 'raw_prices.npy', raw_prices)
safe_write_npy(DIRS['data'] / 'raw_volumes.npy', raw_volumes)
safe_write_npy(DIRS['data'] / 'regime_state.npy', regime_state)
safe_write_npy(DIRS['data'] / 'corp_actions.npy', corp_actions)

# Data quality report
num_missing = np.sum(missing_mask)
total_points = T * N
missing_rate = num_missing / total_points

data_quality = {
    'total_data_points': int(total_points),
    'missing_points': int(num_missing),
    'missing_rate': float(missing_rate),
    'time_monotonic': True,
    'price_range': [float(np.nanmin(raw_prices)), float(np.nanmax(raw_prices))],
    'num_corp_actions': int(np.sum(corp_actions)),
}
safe_write_json(DIRS['data'] / 'data_quality.json', data_quality)

# Data fingerprint
data_fingerprint = {
    'config_hash': config_hash,
    'raw_prices_hash': sha256_file(DIRS['data'] / 'raw_prices.npy'),
    'raw_volumes_hash': sha256_file(DIRS['data'] / 'raw_volumes.npy'),
    'regime_state_hash': sha256_file(DIRS['data'] / 'regime_state.npy'),
    'corp_actions_hash': sha256_file(DIRS['data'] / 'corp_actions.npy'),
}
safe_write_json(DIRS['data'] / 'data_fingerprint.json', data_fingerprint)

# Register artifacts
add_artifact('universe_manifest', DIRS['data'] / 'universe_manifest.json',
             sha256_file(DIRS['data'] / 'universe_manifest.json'), 'v1.0', ['config'])
add_artifact('raw_prices', DIRS['data'] / 'raw_prices.npy',
             data_fingerprint['raw_prices_hash'], 'v1.0', ['config'])
add_artifact('raw_volumes', DIRS['data'] / 'raw_volumes.npy',
             data_fingerprint['raw_volumes_hash'], 'v1.0', ['config'])
add_artifact('regime_state', DIRS['data'] / 'regime_state.npy',
             data_fingerprint['regime_state_hash'], 'v1.0', ['config'])
add_artifact('corp_actions', DIRS['data'] / 'corp_actions.npy',
             data_fingerprint['corp_actions_hash'], 'v1.0', ['config'])

print("=== SYNTHETIC DATA GENERATED ===")
print(f"Universe: {N} assets, {T} bars")
print(f"Tickers: {tickers[:5]} ... {tickers[-1]}")
print(f"Price range: [{data_quality['price_range'][0]:.2f}, {data_quality['price_range'][1]:.2f}]")
print(f"Missing data: {num_missing}/{total_points} ({missing_rate:.2%})")
print(f"Corporate actions: {data_quality['num_corp_actions']}")
print(f"Regime switches: {np.sum(np.diff(regime_state) != 0)}")
print()

# Cell 5 — Data normalization and alignment (survivorship-safe, simple)

"""
Data Normalization and Alignment

Handles:
- Corporate action adjustments (simplified: adjust prices backward)
- Missing data (forward fill for simplicity)
- Return calculation (log returns)
- Universe alignment (constant universe in this case)

Key principle: All adjustments preserve causality.
"""

# Load data
raw_prices = np.load(DIRS['data'] / 'raw_prices.npy')
corp_actions = np.load(DIRS['data'] / 'corp_actions.npy')

# Corporate action policy: Simple adjustment
# When a corporate action occurs at time t, we adjust all prices before t
# This is a simplified model - in production you'd have detailed adjustment factors
corp_actions_policy = {
    'method': 'backward_adjustment',
    'description': 'When corp action at time t, multiply all prior prices by adjustment factor',
    'default_adjustment_factor': 0.98,  # Simplified: assume 2% adjustment
}

# Apply corporate action adjustments
adj_prices = raw_prices.copy()

for i in range(N):
    corp_action_times = np.where(corp_actions[:, i] == 1)[0]
    for t_action in corp_action_times:
        if t_action > 0:
            # Adjust all prices before t_action
            adj_prices[:t_action, i] *= corp_actions_policy['default_adjustment_factor']

# Handle missing data: forward fill
# This is causal because we only use past values
for i in range(N):
    for t in range(1, T):
        if np.isnan(adj_prices[t, i]) and not np.isnan(adj_prices[t-1, i]):
            adj_prices[t, i] = adj_prices[t-1, i]

# If first value is NaN, backfill from first non-NaN
for i in range(N):
    if np.isnan(adj_prices[0, i]):
        first_valid = np.where(~np.isnan(adj_prices[:, i]))[0]
        if len(first_valid) > 0:
            adj_prices[0, i] = adj_prices[first_valid[0], i]

# Calculate returns (log returns)
returns = np.zeros((T, N))
returns[0, :] = 0  # No return at t=0

for t in range(1, T):
    with np.errstate(divide='ignore', invalid='ignore'):
        returns[t, :] = np.log(adj_prices[t, :] / adj_prices[t-1, :])

    # Handle any remaining NaN or inf
    returns[t, np.isnan(returns[t, :])] = 0
    returns[t, np.isinf(returns[t, :])] = 0

# Save artifacts
safe_write_json(DIRS['data'] / 'corp_actions_policy.json', corp_actions_policy)
safe_write_npy(DIRS['data'] / 'adj_prices.npy', adj_prices)
safe_write_npy(DIRS['data'] / 'returns.npy', returns)

# Alignment proof: document that universe is constant
alignment_proof = {
    'universe_type': 'constant',
    'num_assets': N,
    'num_bars': T,
    'all_assets_present': True,
    'forward_fill_applied': True,
    'return_calculation': 'log',
}
safe_write_json(DIRS['data'] / 'alignment_proof.json', alignment_proof)

# Adjusted data fingerprint
adj_data_fingerprint = {
    'depends_on': ['raw_prices', 'corp_actions'],
    'adj_prices_hash': sha256_file(DIRS['data'] / 'adj_prices.npy'),
    'returns_hash': sha256_file(DIRS['data'] / 'returns.npy'),
    'policy_hash': sha256_json(corp_actions_policy),
}
safe_write_json(DIRS['data'] / 'adj_data_fingerprint.json', adj_data_fingerprint)

# Register artifacts
add_artifact('adj_prices', DIRS['data'] / 'adj_prices.npy',
             adj_data_fingerprint['adj_prices_hash'], 'v1.0', ['raw_prices', 'corp_actions'])
add_artifact('returns', DIRS['data'] / 'returns.npy',
             adj_data_fingerprint['returns_hash'], 'v1.0', ['adj_prices'])

print("=== DATA NORMALIZED ===")
print(f"Adjusted prices shape: {adj_prices.shape}")
print(f"Returns shape: {returns.shape}")
print(f"Returns range: [{np.min(returns):.4f}, {np.max(returns):.4f}]")
print(f"Mean return: {np.mean(returns):.6f}")
print(f"Std return: {np.std(returns):.6f}")
print()


# Cell 6 — Feature engineering with strict causality gates

"""
Feature Engineering with Strict Causality

Implements feature families with explicit causality checks:
- Momentum: rolling return over lookback period, lagged
- Volatility: rolling standard deviation, lagged
- Mean reversion: negative recent return, lagged
- Cross-sectional z-score: standardization at each time t

Key principle: Features at time t use ONLY data up to time t-lag.
"""

# Load returns
returns = np.load(DIRS['data'] / 'returns.npy')

# Extract feature config
mom_lookback = CONFIG['feature']['momentum_lookback']
vol_lookback = CONFIG['feature']['volatility_lookback']
mr_lookback = CONFIG['feature']['mean_reversion_lookback']
lag = CONFIG['feature']['causality_lag']

# Number of features
num_features = 4  # momentum, volatility, mean_reversion, zscore_momentum
features = np.zeros((T, N, num_features))

def rolling_sum(x, window):
    """Compute rolling sum over window. Returns array of same length, with NaN for initial values."""
    result = np.full_like(x, np.nan)
    for i in range(window-1, len(x)):
        result[i] = np.sum(x[i-window+1:i+1])
    return result

def rolling_mean(x, window):
    """Compute rolling mean over window."""
    result = np.full_like(x, np.nan)
    for i in range(window-1, len(x)):
        result[i] = np.mean(x[i-window+1:i+1])
    return result

def rolling_std(x, window):
    """Compute rolling standard deviation over window."""
    result = np.full_like(x, np.nan)
    for i in range(window-1, len(x)):
        result[i] = np.std(x[i-window+1:i+1], ddof=1)
    return result

# Feature 0: Momentum (cumulative return over lookback, lagged by 1)
for i in range(N):
    momentum_raw = rolling_sum(returns[:, i], mom_lookback)
    # Lag by 1: feature at time t uses data up to t-1
    features[lag:, i, 0] = momentum_raw[:-lag] if lag > 0 else momentum_raw

# Feature 1: Volatility (rolling std, lagged by 1)
for i in range(N):
    vol_raw = rolling_std(returns[:, i], vol_lookback)
    features[lag:, i, 1] = vol_raw[:-lag] if lag > 0 else vol_raw

# Feature 2: Mean reversion (negative of recent return, lagged)
for i in range(N):
    mr_raw = -rolling_sum(returns[:, i], mr_lookback)
    features[lag:, i, 2] = mr_raw[:-lag] if lag > 0 else mr_raw

# Feature 3: Cross-sectional z-score of momentum
# At each time t, standardize momentum feature across assets
# This uses only values at time t (which are already lagged)
if CONFIG['feature']['cross_sectional_zscore']:
    for t in range(T):
        mom_t = features[t, :, 0]
        valid = ~np.isnan(mom_t)
        if np.sum(valid) > 1:
            mean_t = np.mean(mom_t[valid])
            std_t = np.std(mom_t[valid], ddof=1)
            if std_t > 1e-8:
                features[t, valid, 3] = (mom_t[valid] - mean_t) / std_t

# Replace NaN with 0 for simplicity (or could use more sophisticated handling)
features[np.isnan(features)] = 0

# Causality check: verify features at time t use only data up to t-lag
leakage_tests = {
    'feature_lag': int(lag),
    'checks': []
}

for feat_idx, feat_name in enumerate(['momentum', 'volatility', 'mean_reversion', 'zscore_momentum']):
    # Check that feature at time t does not use return at time t
    # We verify by checking that first non-zero feature appears after appropriate lag
    for i in range(N):
        first_nonzero = np.where(features[:, i, feat_idx] != 0)[0]
        if len(first_nonzero) > 0:
            min_time = int(first_nonzero[0])
            expected_min = int(lag)  # Features should start at time lag or later

            leakage_tests['checks'].append({
                'feature': feat_name,
                'asset': int(i),
                'first_nonzero_time': min_time,
                'expected_min_time': expected_min,
                'pass': bool(min_time >= expected_min),  # Convert to Python bool
            })

all_passed = all(check['pass'] for check in leakage_tests['checks'])
leakage_tests['all_passed'] = bool(all_passed)  # Convert to Python bool

# Save artifacts
safe_write_npy(DIRS['features'] / 'features.npy', features)

feature_manifest = {
    'num_features': num_features,
    'feature_names': ['momentum', 'volatility', 'mean_reversion', 'zscore_momentum'],
    'shape': list(features.shape),
    'config': {
        'momentum_lookback': mom_lookback,
        'volatility_lookback': vol_lookback,
        'mean_reversion_lookback': mr_lookback,
        'causality_lag': lag,
    },
}
safe_write_json(DIRS['features'] / 'feature_manifest.json', feature_manifest)
safe_write_json(DIRS['features'] / 'leakage_tests.json', leakage_tests)

feature_fingerprint = {
    'depends_on': ['returns'],
    'features_hash': sha256_file(DIRS['features'] / 'features.npy'),
    'manifest_hash': sha256_json(feature_manifest),
}
safe_write_json(DIRS['features'] / 'feature_fingerprint.json', feature_fingerprint)

# Register artifacts
add_artifact('features', DIRS['features'] / 'features.npy',
             feature_fingerprint['features_hash'], 'v1.0', ['returns'])
add_artifact('feature_manifest', DIRS['features'] / 'feature_manifest.json',
             feature_fingerprint['manifest_hash'], 'v1.0', ['config'])

print("=== FEATURES GENERATED ===")
print(f"Features shape: {features.shape}")
print(f"Feature names: {feature_manifest['feature_names']}")
print(f"Causality tests: {len(leakage_tests['checks'])} checks")
print(f"All causality tests passed: {all_passed}")
if not all_passed:
    failed = [c for c in leakage_tests['checks'] if not c['pass']]
    print(f"WARNING: {len(failed)} causality tests failed!")
print()






# Cell 7 — Baseline signals (Ch.7–10) as modules

"""
Baseline Signal Generation

Creates modular signals from features:
- Trend signal: based on momentum feature
- Mean reversion signal: based on mean reversion feature
- Volatility input: used for risk scaling

Each signal is a transformation of features into actionable scores.
"""

# Load features
features = np.load(DIRS['features'] / 'features.npy')

# Signal functions
def trend_signal(momentum_feature):
    """
    Trend signal: positive momentum -> positive signal
    Simple linear scaling with clipping
    """
    signal = np.clip(momentum_feature * 10, -1, 1)
    return signal

def mean_reversion_signal(mr_feature):
    """
    Mean reversion signal: negative recent return -> positive signal
    (mr_feature is already negative of recent return)
    """
    signal = np.clip(mr_feature * 20, -1, 1)
    return signal

def volatility_input(vol_feature):
    """
    Volatility for risk scaling (not a directional signal)
    Returns inverse volatility for weighting
    """
    # Avoid division by zero
    vol_safe = np.where(vol_feature > 1e-6, vol_feature, 1e-6)
    return 1.0 / vol_safe

# Generate baseline signals
signals_baseline = np.zeros((T, N, 3))

if CONFIG['signal']['trend_enabled']:
    signals_baseline[:, :, 0] = trend_signal(features[:, :, 0])  # momentum feature

if CONFIG['signal']['mean_reversion_enabled']:
    signals_baseline[:, :, 1] = mean_reversion_signal(features[:, :, 2])  # MR feature

if CONFIG['signal']['volatility_targeting_enabled']:
    signals_baseline[:, :, 2] = volatility_input(features[:, :, 1])  # volatility feature

# Signal diagnostics
signal_names = ['trend', 'mean_reversion', 'inv_volatility']
signal_diagnostics = {
    'signals': []
}

for idx, name in enumerate(signal_names):
    sig = signals_baseline[:, :, idx]

    diagnostics = {
        'name': name,
        'mean': float(np.mean(sig)),
        'std': float(np.std(sig)),
        'min': float(np.min(sig)),
        'max': float(np.max(sig)),
        'num_nonzero': int(np.sum(sig != 0)),
        'fraction_nonzero': float(np.sum(sig != 0) / sig.size),
    }

    # Implied turnover proxy: mean absolute change in signal
    changes = np.abs(np.diff(sig, axis=0))
    diagnostics['mean_abs_change'] = float(np.mean(changes))

    signal_diagnostics['signals'].append(diagnostics)

# Save artifacts
safe_write_npy(DIRS['signals'] / 'signals_baseline.npy', signals_baseline)

signal_manifest_baseline = {
    'num_signals': 3,
    'signal_names': signal_names,
    'shape': list(signals_baseline.shape),
    'enabled': {
        'trend': CONFIG['signal']['trend_enabled'],
        'mean_reversion': CONFIG['signal']['mean_reversion_enabled'],
        'volatility_targeting': CONFIG['signal']['volatility_targeting_enabled'],
    },
}
safe_write_json(DIRS['signals'] / 'signal_manifest_baseline.json', signal_manifest_baseline)
safe_write_json(DIRS['signals'] / 'signal_diagnostics_baseline.json', signal_diagnostics)

signal_fingerprint_baseline = {
    'depends_on': ['features'],
    'signals_hash': sha256_file(DIRS['signals'] / 'signals_baseline.npy'),
    'manifest_hash': sha256_json(signal_manifest_baseline),
}
safe_write_json(DIRS['signals'] / 'signal_fingerprint_baseline.json', signal_fingerprint_baseline)

# Register artifacts
add_artifact('signals_baseline', DIRS['signals'] / 'signals_baseline.npy',
             signal_fingerprint_baseline['signals_hash'], 'v1.0', ['features'])

print("=== BASELINE SIGNALS GENERATED ===")
print(f"Signals shape: {signals_baseline.shape}")
print(f"Signal names: {signal_names}")
print("\nSignal diagnostics:")
for diag in signal_diagnostics['signals']:
    print(f"  {diag['name']:20s}: mean={diag['mean']:7.4f}, std={diag['std']:7.4f}, "
          f"turnover_proxy={diag['mean_abs_change']:.4f}")
print()

# Cell 8 — Optional regime routing (Ch.14) with strict timing

"""
Regime Detection and Routing

Detects market regimes using ONLY past data:
- High/low volatility regimes
- Regime state at time t uses returns up to t-1

Routing policy:
- High volatility: reduce trend, increase mean reversion
- Low volatility: increase trend, reduce mean reversion
"""

# Load returns
returns = np.load(DIRS['data'] / 'returns.npy')

# Regime detection
regime_enabled = CONFIG['regime']['enabled']

if regime_enabled:
    lookback = CONFIG['regime']['lookback']
    threshold = CONFIG['regime']['volatility_threshold']

    # Compute rolling volatility using ONLY past returns
    regime_state = np.zeros(T)

    for t in range(lookback, T):
        # Use returns from t-lookback to t-1 (not including t)
        past_returns = returns[t-lookback:t, :]
        rolling_vol = np.std(past_returns)

        # High vol regime if above threshold
        regime_state[t] = 1 if rolling_vol > threshold else 0

    # Routing policy
    routing_policy = {
        'regime_0': {  # Low volatility
            'description': 'Low volatility regime - favor trend',
            'trend_weight': 0.7,
            'mean_reversion_weight': 0.3,
        },
        'regime_1': {  # High volatility
            'description': 'High volatility regime - favor mean reversion',
            'trend_weight': 0.3,
            'mean_reversion_weight': 0.7,
        },
    }

    # Regime diagnostics
    regime_diagnostics = {
        'num_regime_0': int(np.sum(regime_state == 0)),
        'num_regime_1': int(np.sum(regime_state == 1)),
        'fraction_regime_1': float(np.mean(regime_state)),
        'num_switches': int(np.sum(np.abs(np.diff(regime_state)))),
    }

else:
    # Regime disabled: all zeros (default regime)
    regime_state = np.zeros(T)
    routing_policy = {
        'regime_0': {
            'description': 'Regime detection disabled - use default weights',
            'trend_weight': 0.5,
            'mean_reversion_weight': 0.5,
        },
    }
    regime_diagnostics = {
        'enabled': False,
    }

# Save artifacts
safe_write_npy(DIRS['signals'] / 'regime_state.npy', regime_state)
safe_write_json(DIRS['signals'] / 'routing_policy.json', routing_policy)
safe_write_json(DIRS['signals'] / 'regime_diagnostics.json', regime_diagnostics)

# Register artifacts
add_artifact('regime_state', DIRS['signals'] / 'regime_state.npy',
             sha256_file(DIRS['signals'] / 'regime_state.npy'), 'v1.0', ['returns'])

print("=== REGIME DETECTION ===")
print(f"Regime enabled: {regime_enabled}")
if regime_enabled:
    print(f"Regime 0 (low vol): {regime_diagnostics['num_regime_0']} bars")
    print(f"Regime 1 (high vol): {regime_diagnostics['num_regime_1']} bars")
    print(f"Regime switches: {regime_diagnostics['num_switches']}")
else:
    print("Regime detection disabled - using default weights")
print()

# Cell 9 — Signal combination layer (stacking/voting/risk-weighting)

"""
Signal Combination Layer

Combines baseline signals into final combined signal:
- Applies regime-based routing weights
- Applies volatility-based risk weighting
- Logs signal contributions for attribution
"""

# Load signals and regime
signals_baseline = np.load(DIRS['signals'] / 'signals_baseline.npy')
regime_state = np.load(DIRS['signals'] / 'regime_state.npy')
routing_policy = json.load(open(DIRS['signals'] / 'routing_policy.json'))

# Extract individual signals
trend_sig = signals_baseline[:, :, 0]
mr_sig = signals_baseline[:, :, 1]
inv_vol = signals_baseline[:, :, 2]

# Initialize combined signal
signals_combined = np.zeros((T, N))

# Track contributions
signal_contrib = {
    'trend_contribution': np.zeros((T, N)),
    'mean_reversion_contribution': np.zeros((T, N)),
}

# Combine signals with regime routing
for t in range(T):
    regime_t = int(regime_state[t])
    regime_key = f'regime_{regime_t}'

    if regime_key in routing_policy:
        w_trend = routing_policy[regime_key]['trend_weight']
        w_mr = routing_policy[regime_key]['mean_reversion_weight']
    else:
        # Default weights
        w_trend = 0.5
        w_mr = 0.5

    # Combine signals
    combined_raw = w_trend * trend_sig[t, :] + w_mr * mr_sig[t, :]

    # Track contributions
    signal_contrib['trend_contribution'][t, :] = w_trend * trend_sig[t, :]
    signal_contrib['mean_reversion_contribution'][t, :] = w_mr * mr_sig[t, :]

    # Apply risk weighting (scale by inverse volatility)
    # Normalize inv_vol to avoid extreme scaling
    inv_vol_t = inv_vol[t, :]
    inv_vol_norm = inv_vol_t / (np.mean(inv_vol_t) + 1e-8)

    signals_combined[t, :] = combined_raw * inv_vol_norm

# Normalize combined signal to [-1, 1] range
signals_combined = np.clip(signals_combined, -2, 2) / 2

# Save artifacts
safe_write_npy(DIRS['signals'] / 'signals_combined.npy', signals_combined)

combination_manifest = {
    'method': 'regime_routing_with_risk_weighting',
    'routing_policy_applied': True,
    'risk_weighting_applied': True,
    'output_range': [-1.0, 1.0],
}
safe_write_json(DIRS['signals'] / 'combination_manifest.json', combination_manifest)

# Save contributions (subset for efficiency)
signal_contrib_summary = {
    'mean_trend_contribution': float(np.mean(signal_contrib['trend_contribution'])),
    'mean_mr_contribution': float(np.mean(signal_contrib['mean_reversion_contribution'])),
    'trend_dominance_fraction': float(np.mean(
        np.abs(signal_contrib['trend_contribution']) >
        np.abs(signal_contrib['mean_reversion_contribution'])
    )),
}
safe_write_json(DIRS['signals'] / 'signal_contrib.json', signal_contrib_summary)

# Register artifacts
add_artifact('signals_combined', DIRS['signals'] / 'signals_combined.npy',
             sha256_file(DIRS['signals'] / 'signals_combined.npy'), 'v1.0',
             ['signals_baseline', 'regime_state'])

print("=== SIGNALS COMBINED ===")
print(f"Combined signals shape: {signals_combined.shape}")
print(f"Signal range: [{np.min(signals_combined):.4f}, {np.max(signals_combined):.4f}]")
print(f"Mean trend contribution: {signal_contrib_summary['mean_trend_contribution']:.4f}")
print(f"Mean MR contribution: {signal_contrib_summary['mean_mr_contribution']:.4f}")
print(f"Trend dominance: {signal_contrib_summary['trend_dominance_fraction']:.2%}")
print()

# Cell 10 — Portfolio construction (Ch.16) under constraints

"""
Portfolio Construction Under Constraints

Maps signals to portfolio weights with:
- Mean-variance optimization (simplified heuristic)
- Position limits (max weight per asset)
- Leverage constraint (L1 norm)
- Dollar neutrality (sum = 0)

Optimization method: heuristic mean-variance
- Convert signals to expected returns (mu)
- Use diagonal risk model (variance from vol feature)
- Compute raw weights = mu / variance
- Apply constraints iteratively
"""

# Load signals and features
signals_combined = np.load(DIRS['signals'] / 'signals_combined.npy')
features = np.load(DIRS['features'] / 'features.npy')

# Extract config
max_leverage = CONFIG['portfolio']['max_leverage']
max_weight = CONFIG['portfolio']['max_weight_per_asset']
neutrality = CONFIG['portfolio']['neutrality_constraint']

# Initialize target weights
weights_target = np.zeros((T, N))

# Volatility feature for risk model (feature index 1)
vol_feature = features[:, :, 1]

# Solver trace for diagnostics
solver_trace = {
    'num_clipped': [],
    'num_rescaled': [],
    'leverage_before': [],
    'leverage_after': [],
}

for t in range(T):
    # Expected return proxy: signal
    mu = signals_combined[t, :]

    # Variance estimate
    var = vol_feature[t, :] ** 2
    var = np.where(var > 1e-8, var, 1e-8)  # Avoid division by zero

    # Raw weights: mu / variance (mean-variance optimal)
    weights_raw = mu / var

    # Apply per-asset constraint
    weights_clipped = np.clip(weights_raw, -max_weight, max_weight)
    num_clipped = np.sum(np.abs(weights_clipped) != np.abs(weights_raw))

    # Apply leverage constraint (L1 norm)
    leverage_before = np.sum(np.abs(weights_clipped))
    if leverage_before > max_leverage:
        weights_clipped = weights_clipped * (max_leverage / leverage_before)
    leverage_after = np.sum(np.abs(weights_clipped))

    # Apply neutrality constraint
    if neutrality == 'dollar_neutral':
        # Ensure sum = 0 by removing mean
        weights_clipped = weights_clipped - np.mean(weights_clipped)

    # Final rescale to ensure leverage constraint after neutrality adjustment
    leverage_final = np.sum(np.abs(weights_clipped))
    if leverage_final > max_leverage:
        weights_clipped = weights_clipped * (max_leverage / leverage_final)

    weights_target[t, :] = weights_clipped

    # Log solver trace
    solver_trace['num_clipped'].append(int(num_clipped))
    solver_trace['num_rescaled'].append(1 if leverage_before > max_leverage else 0)
    solver_trace['leverage_before'].append(float(leverage_before))
    solver_trace['leverage_after'].append(float(leverage_after))

# Aggregate solver trace
solver_trace_summary = {
    'mean_clipped': float(np.mean(solver_trace['num_clipped'])),
    'mean_rescaled': float(np.mean(solver_trace['num_rescaled'])),
    'mean_leverage_before': float(np.mean(solver_trace['leverage_before'])),
    'mean_leverage_after': float(np.mean(solver_trace['leverage_after'])),
}

# Save artifacts
safe_write_npy(DIRS['portfolio'] / 'weights_target.npy', weights_target)

portfolio_manifest = {
    'method': 'heuristic_mean_variance',
    'risk_model': 'diagonal',
    'constraints': {
        'max_leverage': max_leverage,
        'max_weight_per_asset': max_weight,
        'neutrality': neutrality,
    },
}
safe_write_json(DIRS['portfolio'] / 'portfolio_manifest.json', portfolio_manifest)

constraints_bound = {
    'max_leverage_bound': float(np.max([np.sum(np.abs(weights_target[t, :])) for t in range(T)])),
    'max_weight_bound': float(np.max(np.abs(weights_target))),
    'neutrality_error': float(np.max([np.abs(np.sum(weights_target[t, :])) for t in range(T)])),
}
safe_write_json(DIRS['portfolio'] / 'constraints_bound.json', constraints_bound)
safe_write_json(DIRS['portfolio'] / 'solver_trace.json', solver_trace_summary)

# Register artifacts
add_artifact('weights_target', DIRS['portfolio'] / 'weights_target.npy',
             sha256_file(DIRS['portfolio'] / 'weights_target.npy'), 'v1.0',
             ['signals_combined', 'features'])

print("=== PORTFOLIO CONSTRUCTED ===")
print(f"Weights shape: {weights_target.shape}")
print(f"Max leverage: {constraints_bound['max_leverage_bound']:.4f} (limit: {max_leverage})")
print(f"Max weight: {constraints_bound['max_weight_bound']:.4f} (limit: {max_weight})")
print(f"Neutrality error: {constraints_bound['neutrality_error']:.6f}")
print(f"Avg positions clipped: {solver_trace_summary['mean_clipped']:.1f} per bar")
print(f"Avg leverage rescale: {solver_trace_summary['mean_rescaled']:.2%} of bars")
print()

# Cell 11 — Risk overlays (Ch.10 & Ch.17)

"""
Risk Overlays: Dynamic Risk Management

Implements:
- Volatility targeting: scale portfolio to target volatility
- Drawdown control: reduce exposure during drawdowns
- Circuit breaker: emergency stop on large losses

All overlays preserve causality by using only past data.
"""

# Load weights and returns
weights_target = np.load(DIRS['portfolio'] / 'weights_target.npy')
returns = np.load(DIRS['data'] / 'returns.npy')

# Extract config
vol_target = CONFIG['risk']['volatility_target']
vol_lookback = CONFIG['risk']['volatility_target_lookback']
dd_threshold = CONFIG['risk']['drawdown_threshold']
dd_response = CONFIG['risk']['drawdown_response']
dd_scale = CONFIG['risk']['drawdown_scale_factor']
circuit_breaker_enabled = CONFIG['risk']['circuit_breaker_enabled']
circuit_breaker_threshold = CONFIG['risk']['circuit_breaker_loss_threshold']

# Initialize final weights
weights_final = weights_target.copy()

# Track risk events
risk_events = {
    'volatility_adjustments': [],
    'drawdown_events': [],
    'circuit_breaker_events': [],
}

# Compute portfolio returns (before risk overlays) to compute realized vol
portfolio_returns_pre_risk = np.zeros(T)
for t in range(1, T):
    # Use weights from t-1 applied to returns at t
    portfolio_returns_pre_risk[t] = np.sum(weights_target[t-1, :] * returns[t, :])

# Volatility targeting
for t in range(vol_lookback, T):
    # Realized volatility using past returns
    realized_vol = np.std(portfolio_returns_pre_risk[t-vol_lookback:t])

    if realized_vol > 1e-6:
        # Scale to match target (daily target = annual / sqrt(252))
        daily_target = vol_target / np.sqrt(252)
        vol_scale = daily_target / realized_vol

        # Clip scaling to reasonable range [0.5, 2.0]
        vol_scale = np.clip(vol_scale, 0.5, 2.0)

        weights_final[t, :] = weights_target[t, :] * vol_scale

        if abs(vol_scale - 1.0) > 0.1:  # Log significant adjustments
            risk_events['volatility_adjustments'].append({
                'time': int(t),
                'realized_vol': float(realized_vol),
                'scale': float(vol_scale),
            })

# Drawdown control
# Compute cumulative returns to track equity curve
equity_curve = np.cumprod(1 + portfolio_returns_pre_risk)
running_max = np.maximum.accumulate(equity_curve)
drawdown = (equity_curve - running_max) / running_max

for t in range(T):
    if drawdown[t] < -dd_threshold:
        if dd_response == 'scale_down':
            weights_final[t, :] *= dd_scale
        elif dd_response == 'go_flat':
            weights_final[t, :] = 0

        risk_events['drawdown_events'].append({
            'time': int(t),
            'drawdown': float(drawdown[t]),
            'action': dd_response,
        })

# Circuit breaker
if circuit_breaker_enabled:
    for t in range(1, T):
        if portfolio_returns_pre_risk[t] < -circuit_breaker_threshold:
            # Emergency stop: go flat for next bar
            if t + 1 < T:
                weights_final[t+1, :] = 0

            risk_events['circuit_breaker_events'].append({
                'time': int(t),
                'loss': float(portfolio_returns_pre_risk[t]),
                'action': 'go_flat_next_bar',
            })

# Save artifacts
safe_write_npy(DIRS['risk'] / 'weights_final.npy', weights_final)

risk_manifest = {
    'overlays_applied': {
        'volatility_targeting': True,
        'drawdown_control': True,
        'circuit_breaker': circuit_breaker_enabled,
    },
    'config': {
        'volatility_target': vol_target,
        'drawdown_threshold': dd_threshold,
        'circuit_breaker_threshold': circuit_breaker_threshold,
    },
}
safe_write_json(DIRS['risk'] / 'risk_manifest.json', risk_manifest)
safe_write_json(DIRS['risk'] / 'risk_events.json', risk_events)

kill_switch = {
    'enabled': circuit_breaker_enabled,
    'num_triggers': len(risk_events['circuit_breaker_events']),
    'triggers': risk_events['circuit_breaker_events'][:5],  # First 5 for brevity
}
safe_write_json(DIRS['risk'] / 'kill_switch.json', kill_switch)

# Register artifacts
add_artifact('weights_final', DIRS['risk'] / 'weights_final.npy',
             sha256_file(DIRS['risk'] / 'weights_final.npy'), 'v1.0',
             ['weights_target'])

print("=== RISK OVERLAYS APPLIED ===")
print(f"Volatility adjustments: {len(risk_events['volatility_adjustments'])}")
print(f"Drawdown events: {len(risk_events['drawdown_events'])}")
print(f"Circuit breaker triggers: {len(risk_events['circuit_breaker_events'])}")
print()

# Cell 12 — Execution + transaction cost model (Ch.18)

"""
Execution and Transaction Costs

Simulates realistic execution with:
- Execution delay (decisions at t, execute at t+delay)
- Transaction costs:
  * Spread cost
  * Commission fees
  * Market impact (proportional to trade size / volume)
- Turnover tracking and caps

Key: Costs are applied at execution time using correct timing.
"""

# Load weights and data
weights_final = np.load(DIRS['risk'] / 'weights_final.npy')
returns = np.load(DIRS['data'] / 'returns.npy')
adj_prices = np.load(DIRS['data'] / 'adj_prices.npy')
raw_volumes = np.load(DIRS['data'] / 'raw_volumes.npy')

# Extract config
exec_delay = CONFIG['time_semantics']['execution_delay_bars']
spread_bps = CONFIG['execution']['spread_bps']
fee_bps = CONFIG['execution']['fee_bps']
impact_coeff = CONFIG['execution']['impact_coefficient']
turnover_cap_annual = CONFIG['execution']['turnover_cap_annual']
turnover_cap_daily = turnover_cap_annual / 252

# Initialize execution arrays
orders = np.zeros((T, N))  # Orders placed at time t (to be executed at t+delay)
fills = np.zeros((T, N))  # Actual fills executed
costs = np.zeros((T, 4))  # [total, spread, fees, impact]

# Track holdings over time
holdings = np.zeros((T, N))

# Compute orders and fills
for t in range(T):
    # Target holdings at time t
    target_holdings_t = weights_final[t, :]

    # Current holdings (from previous period)
    if t > 0:
        current_holdings = holdings[t-1, :]
    else:
        current_holdings = np.zeros(N)

    # Order = target - current
    orders[t, :] = target_holdings_t - current_holdings

    # Execute orders placed exec_delay bars ago
    if t >= exec_delay and t > 0:
        orders_to_execute = orders[t - exec_delay, :]

        # Simulate fills (assume 100% fill rate for simplicity)
        fills[t, :] = orders_to_execute

        # Compute costs
        # Spread cost
        spread_cost = (spread_bps / 10000) * np.sum(np.abs(fills[t, :]))

        # Commission fees
        fee_cost = (fee_bps / 10000) * np.sum(np.abs(fills[t, :]))

        # Market impact: proportional to trade / volume
        # Use volume at execution time
        volume_t = raw_volumes[t, :]
        volume_t = np.where(np.isnan(volume_t), np.nanmean(raw_volumes), volume_t)  # Handle NaN

        # Normalize fills to notional (assuming unit capital)
        impact_cost = impact_coeff * np.sum(np.abs(fills[t, :]) / (volume_t / np.mean(volume_t) + 1e-6))
        impact_cost = min(impact_cost, 0.01)  # Cap impact at 1%

        costs[t, 0] = spread_cost + fee_cost + impact_cost
        costs[t, 1] = spread_cost
        costs[t, 2] = fee_cost
        costs[t, 3] = impact_cost

    # Update holdings (fills get added to holdings)
    if t > 0:
        holdings[t, :] = holdings[t-1, :] + fills[t, :]

# Compute turnover
turnover = np.sum(np.abs(fills), axis=1)
turnover_series = {
    'total': turnover.tolist(),
    'mean': float(np.mean(turnover)),
    'max': float(np.max(turnover)),
    'exceeded_cap': int(np.sum(turnover > turnover_cap_daily)),
}

# Apply turnover cap (post-hoc for this simple model)
# In production, you'd adjust orders before execution
for t in range(T):
    if turnover[t] > turnover_cap_daily:
        # Scale down fills
        scale = turnover_cap_daily / turnover[t]
        fills[t, :] *= scale
        costs[t, :] *= scale

# Save artifacts
safe_write_npy(DIRS['execution'] / 'orders.npy', orders)
safe_write_npy(DIRS['execution'] / 'fills.npy', fills)
safe_write_npy(DIRS['execution'] / 'costs.npy', costs)

exec_manifest = {
    'execution_delay_bars': exec_delay,
    'cost_model': {
        'spread_bps': spread_bps,
        'fee_bps': fee_bps,
        'impact_coefficient': impact_coeff,
    },
    'turnover_cap_daily': turnover_cap_daily,
}
safe_write_json(DIRS['execution'] / 'exec_manifest.json', exec_manifest)
safe_write_json(DIRS['execution'] / 'turnover.json', turnover_series)

# Register artifacts
add_artifact('fills', DIRS['execution'] / 'fills.npy',
             sha256_file(DIRS['execution'] / 'fills.npy'), 'v1.0',
             ['weights_final'])

print("=== EXECUTION SIMULATED ===")
print(f"Mean turnover: {turnover_series['mean']:.4f}")
print(f"Max turnover: {turnover_series['max']:.4f} (cap: {turnover_cap_daily:.4f})")
print(f"Days exceeding cap: {turnover_series['exceeded_cap']}")
print(f"Mean total cost: {np.mean(costs[:, 0]):.6f}")
print(f"  Spread: {np.mean(costs[:, 1]):.6f}")
print(f"  Fees: {np.mean(costs[:, 2]):.6f}")
print(f"  Impact: {np.mean(costs[:, 3]):.6f}")
print()

# Cell 13 — System backtest (Ch.6) with correct timing and benchmarking

"""
System Backtest: Performance Evaluation

Computes:
- Gross returns (before costs)
- Net returns (after costs)
- Benchmark (equal-weight buy-and-hold)
- Performance metrics (CAGR, vol, Sharpe, max drawdown)
- Attribution (signal contributions)
"""

# Load data
returns = np.load(DIRS['data'] / 'returns.npy')
weights_final = np.load(DIRS['risk'] / 'weights_final.npy')
fills = np.load(DIRS['execution'] / 'fills.npy')
costs = np.load(DIRS['execution'] / 'costs.npy')

# Compute gross returns
# At time t, use holdings from t-1 (which includes fills from t-1) applied to returns at t
gross_returns = np.zeros(T)
holdings = np.zeros((T, N))

for t in range(T):
    if t > 0:
        # Holdings at start of period t = previous holdings + fills from previous period
        holdings[t, :] = holdings[t-1, :] + fills[t-1, :]

        # Gross return = holdings * returns
        gross_returns[t] = np.sum(holdings[t, :] * returns[t, :])

# Compute net returns (subtract costs)
net_returns = gross_returns - costs[:, 0]

# Benchmark: equal-weight buy-and-hold
benchmark_weights = np.ones(N) / N
benchmark_returns = np.sum(benchmark_weights * returns, axis=1)

# Compute equity curves
gross_equity = np.cumprod(1 + gross_returns)
net_equity = np.cumprod(1 + net_returns)
benchmark_equity = np.cumprod(1 + benchmark_returns)

# Performance metrics
def compute_metrics(returns_series, name):
    """Compute standard performance metrics."""
    total_return = np.prod(1 + returns_series) - 1
    num_years = len(returns_series) / 252  # Assuming daily bars
    cagr = (1 + total_return) ** (1 / num_years) - 1 if num_years > 0 else 0

    vol = np.std(returns_series) * np.sqrt(252)
    sharpe = (np.mean(returns_series) * 252) / (vol + 1e-8)

    equity = np.cumprod(1 + returns_series)
    running_max = np.maximum.accumulate(equity)
    drawdown = (equity - running_max) / running_max
    max_drawdown = np.min(drawdown)

    return {
        'name': name,
        'total_return': float(total_return),
        'cagr': float(cagr),
        'volatility': float(vol),
        'sharpe': float(sharpe),
        'max_drawdown': float(max_drawdown),
    }

metrics_gross = compute_metrics(gross_returns, 'gross')
metrics_net = compute_metrics(net_returns, 'net')
metrics_benchmark = compute_metrics(benchmark_returns, 'benchmark')

backtest_report = {
    'gross': metrics_gross,
    'net': metrics_net,
    'benchmark': metrics_benchmark,
    'cost_impact': {
        'mean_cost': float(np.mean(costs[:, 0])),
        'total_cost': float(np.sum(costs[:, 0])),
        'return_drag': float(metrics_gross['cagr'] - metrics_net['cagr']),
    },
}

# Attribution (simplified: use signal contributions from earlier)
signal_contrib = json.load(open(DIRS['signals'] / 'signal_contrib.json'))
attribution = {
    'signal_contributions': signal_contrib,
    'note': 'Full attribution requires position-level tracking; this is a simplified version',
}

# Failure modes: identify problem periods
failure_modes = {
    'large_drawdowns': [],
    'high_turnover': [],
    'high_costs': [],
}

equity = net_equity
running_max = np.maximum.accumulate(equity)
drawdown = (equity - running_max) / running_max

for t in range(T):
    if drawdown[t] < -0.10:  # Drawdown > 10%
        failure_modes['large_drawdowns'].append({
            'time': int(t),
            'drawdown': float(drawdown[t]),
        })

# Limit to first 10 events
for key in failure_modes:
    failure_modes[key] = failure_modes[key][:10]

# Save artifacts
safe_write_npy(DIRS['reports'] / 'backtest_equity.npy',
               np.column_stack([gross_equity, net_equity, benchmark_equity]))
safe_write_json(DIRS['reports'] / 'backtest_report.json', backtest_report)
safe_write_json(DIRS['reports'] / 'attribution.json', attribution)
safe_write_json(DIRS['reports'] / 'failure_modes.json', failure_modes)

# Register artifacts
add_artifact('backtest_report', DIRS['reports'] / 'backtest_report.json',
             sha256_json(backtest_report), 'v1.0',
             ['fills', 'costs', 'returns'])

# Plot equity curves
plt.figure(figsize=(12, 6))
plt.plot(gross_equity, label='Gross', linewidth=2)
plt.plot(net_equity, label='Net', linewidth=2)
plt.plot(benchmark_equity, label='Benchmark', linewidth=2, linestyle='--')
plt.xlabel('Time (bars)')
plt.ylabel('Equity')
plt.title('Backtest Equity Curves')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(DIRS['reports'] / 'equity_curves.png', dpi=150)
plt.close()

print("=== BACKTEST RESULTS ===")
print(f"\nGross Performance:")
print(f"  CAGR: {metrics_gross['cagr']:.2%}")
print(f"  Volatility: {metrics_gross['volatility']:.2%}")
print(f"  Sharpe: {metrics_gross['sharpe']:.2f}")
print(f"  Max Drawdown: {metrics_gross['max_drawdown']:.2%}")

print(f"\nNet Performance:")
print(f"  CAGR: {metrics_net['cagr']:.2%}")
print(f"  Volatility: {metrics_net['volatility']:.2%}")
print(f"  Sharpe: {metrics_net['sharpe']:.2f}")
print(f"  Max Drawdown: {metrics_net['max_drawdown']:.2%}")

print(f"\nBenchmark Performance:")
print(f"  CAGR: {metrics_benchmark['cagr']:.2%}")
print(f"  Volatility: {metrics_benchmark['volatility']:.2%}")
print(f"  Sharpe: {metrics_benchmark['sharpe']:.2f}")
print(f"  Max Drawdown: {metrics_benchmark['max_drawdown']:.2%}")

print(f"\nCost Impact:")
print(f"  Return drag: {backtest_report['cost_impact']['return_drag']:.2%}")
print(f"  Total costs: {backtest_report['cost_impact']['total_cost']:.4f}")
print()


# Cell 14 — Minimal test suite (end-to-end safety checks)

"""
Test Suite: End-to-End Safety Checks

Implements critical tests:
1. Time monotonicity
2. No-lookahead verification
3. Reproducibility
4. Cost sanity checks
5. Fail-closed testing
"""

test_results = {
    'tests': [],
    'all_passed': True,
}

def add_test_result(name, passed, details=None):
    """Add a test result to the registry."""
    test_results['tests'].append({
        'name': name,
        'passed': bool(passed),  # Convert to Python bool
        'details': details or {},
    })
    if not passed:
        test_results['all_passed'] = False

# Test 1: Time monotonicity
print("Running Test 1: Time monotonicity...")
time_index = np.arange(T)
try:
    assert_monotonic_time(time_index, "time_index")
    add_test_result('time_monotonicity', True)
    print("  ✓ PASSED")
except AssertionError as e:
    add_test_result('time_monotonicity', False, {'error': str(e)})
    print(f"  ✗ FAILED: {e}")

# Test 2: No-lookahead checks at boundaries
print("Running Test 2: No-lookahead verification...")
lookahead_checks = []

# Data → Features boundary
# Features at time t should use returns up to t-1 (with lag=1)
lag = CONFIG['feature']['causality_lag']
for t in range(lag + 1, min(lag + 10, T)):  # Check first few valid times
    # Feature at time t uses returns up to t-lag
    max_input_time = t - lag
    decision_time = t
    try:
        assert_no_lookahead(max_input_time, decision_time, f"feature_at_t={t}")
        lookahead_checks.append(True)
    except AssertionError:
        lookahead_checks.append(False)

all_lookahead_passed = all(lookahead_checks)
add_test_result('no_lookahead_features', all_lookahead_passed,
                {'num_checks': len(lookahead_checks), 'num_passed': sum(lookahead_checks)})
if all_lookahead_passed:
    print("  ✓ PASSED")
else:
    print(f"  ✗ FAILED: {sum(lookahead_checks)}/{len(lookahead_checks)} checks passed")

# Test 3: Reproducibility
print("Running Test 3: Reproducibility...")

# Re-compute config hash and compare
current_config_hash = sha256_json(CONFIG)
stored_config_hash = json.load(open(DIRS['data'] / 'data_fingerprint.json'))['config_hash']

config_hash_match = (current_config_hash == stored_config_hash)
add_test_result('config_hash_reproducibility', config_hash_match,
                {'current': current_config_hash[:16], 'stored': stored_config_hash[:16]})

# Re-compute one artifact hash (e.g., features) and compare
features_current = np.load(DIRS['features'] / 'features.npy')
features_hash_current = sha256_bytes(features_current.tobytes())
features_hash_stored = json.load(open(DIRS['features'] / 'feature_fingerprint.json'))['features_hash']

features_hash_match = (features_hash_current == features_hash_stored)
add_test_result('features_hash_reproducibility', features_hash_match,
                {'current': features_hash_current[:16], 'stored': features_hash_stored[:16]})

reproducibility_passed = config_hash_match and features_hash_match
if reproducibility_passed:
    print("  ✓ PASSED")
else:
    print(f"  ✗ FAILED: config_match={config_hash_match}, features_match={features_hash_match}")

# Test 4: Cost sanity checks
print("Running Test 4: Cost sanity checks...")

# Net <= Gross
net_returns_vals = np.load(DIRS['reports'] / 'backtest_equity.npy')[:, 1]
gross_returns_vals = np.load(DIRS['reports'] / 'backtest_equity.npy')[:, 0]
net_le_gross = bool(np.all(net_returns_vals <= gross_returns_vals + 1e-8))  # Convert to Python bool

# Costs non-negative
costs_all = np.load(DIRS['execution'] / 'costs.npy')
costs_nonneg = bool(np.all(costs_all >= -1e-8))  # Convert to Python bool

# Turnover cap respected (after enforcement)
turnover_data = json.load(open(DIRS['execution'] / 'turnover.json'))
turnover_cap = CONFIG['execution']['turnover_cap_annual'] / 252
turnover_cap_ok = bool(turnover_data['max'] <= turnover_cap + 1e-6)  # Convert to Python bool

cost_sanity_passed = net_le_gross and costs_nonneg and turnover_cap_ok
add_test_result('cost_sanity', cost_sanity_passed, {
    'net_le_gross': net_le_gross,
    'costs_nonneg': costs_nonneg,
    'turnover_cap_ok': turnover_cap_ok,
})
if cost_sanity_passed:
    print("  ✓ PASSED")
else:
    print(f"  ✗ FAILED: net≤gross={net_le_gross}, costs≥0={costs_nonneg}, turnover_ok={turnover_cap_ok}")

# Test 5: Fail-closed test
print("Running Test 5: Fail-closed behavior...")

# Simulate corrupted input: set a feature to NaN and verify system handles gracefully
# We'll test by checking that NaN features were replaced with 0
features_test = np.load(DIRS['features'] / 'features.npy')
has_nan = bool(np.any(np.isnan(features_test)))  # Convert to Python bool

# Also check circuit breaker was triggered if large loss occurred
circuit_breaker_triggered = len(json.load(open(DIRS['risk'] / 'kill_switch.json'))['triggers']) > 0

fail_closed_passed = (not has_nan)  # Features should have no NaN after cleaning
add_test_result('fail_closed', fail_closed_passed, {
    'features_have_nan': has_nan,
    'circuit_breaker_available': CONFIG['risk']['circuit_breaker_enabled'],
    'circuit_breaker_triggered': circuit_breaker_triggered,
})
if fail_closed_passed:
    print("  ✓ PASSED")
else:
    print(f"  ✗ FAILED: features have NaN={has_nan}")

# Save test results
safe_write_json(DIRS['validation'] / 'test_results.json', test_results)

reproducibility_check = {
    'config_hash_match': bool(config_hash_match),
    'features_hash_match': bool(features_hash_match),
    'run_id': RUN_ID,
    'seed': SEED,
}
safe_write_json(DIRS['validation'] / 'reproducibility_check.json', reproducibility_check)

# Register artifacts
add_artifact('test_results', DIRS['validation'] / 'test_results.json',
             sha256_json(test_results), 'v1.0', ['all_prior_artifacts'])

print(f"\n=== TEST SUITE COMPLETE ===")
print(f"Total tests: {len(test_results['tests'])}")
print(f"Passed: {sum(1 for t in test_results['tests'] if t['passed'])}")
print(f"Failed: {sum(1 for t in test_results['tests'] if not t['passed'])}")
print(f"All tests passed: {test_results['all_passed']}")
print()







# Cell 15 — Governance artifacts: acceptance criteria and promotion gate

"""
Governance Artifacts: Acceptance Criteria and Promotion Gate

Defines acceptance criteria for production deployment:
- All causal tests must pass
- Reproducibility must pass
- Net performance must be plausible (positive Sharpe)
- Robustness checks (split sample) must pass

Produces governance pack for audit trail and approvals.
"""

# Load test results and performance
test_results = json.load(open(DIRS['validation'] / 'test_results.json'))
backtest_report = json.load(open(DIRS['reports'] / 'backtest_report.json'))

# Define acceptance criteria
acceptance_criteria = {
    'causal_tests': {
        'required': True,
        'passed': test_results['tests'][1]['passed'],  # no-lookahead test
        'description': 'All causality checks must pass',
    },
    'reproducibility': {
        'required': True,
        'passed': test_results['tests'][2]['passed'] and test_results['tests'][3]['passed'],
        'description': 'Config and artifact hashes must match',
    },
    'net_performance': {
        'required': True,
        'passed': backtest_report['net']['sharpe'] > 0,
        'description': 'Net Sharpe ratio must be positive',
        'value': backtest_report['net']['sharpe'],
    },
    'cost_sanity': {
        'required': True,
        'passed': test_results['tests'][4]['passed'],
        'description': 'Costs must be non-negative, net <= gross',
    },
    'robustness': {
        'required': False,  # Not implemented in this basic version
        'passed': True,  # Placeholder
        'description': 'Split-sample validation (placeholder)',
    },
}

# Overall acceptance status
all_required_passed = all(
    criteria['passed'] for criteria in acceptance_criteria.values() if criteria['required']
)

acceptance_draft = {
    'run_id': RUN_ID,
    'timestamp': datetime.now().isoformat(),
    'criteria': acceptance_criteria,
    'overall_status': 'PASS' if all_required_passed else 'FAIL',
    'artifact_hashes': {
        'config': sha256_json(CONFIG),
        'backtest_report': sha256_json(backtest_report),
        'test_results': sha256_json(test_results),
    },
}

# Governance pack
governance_pack = {
    'run_id': RUN_ID,
    'acceptance_status': acceptance_draft['overall_status'],
    'approvals': {
        'research_lead': {'status': 'pending', 'timestamp': None},
        'risk_officer': {'status': 'pending', 'timestamp': None},
        'compliance': {'status': 'pending', 'timestamp': None},
    },
    'change_log': [
        {
            'version': '1.0.0',
            'date': datetime.now().isoformat(),
            'description': 'Initial capstone system implementation',
            'author': 'system',
        }
    ],
    'audit_trail': {
        'artifact_registry': 'manifest/artifact_registry.json',
        'test_results': 'validation/test_results.json',
        'acceptance_draft': 'governance/acceptance_draft.json',
    },
}

# Monitoring pack
monitoring_pack = {
    'run_id': RUN_ID,
    'metrics_contract': {
        'required_metrics': [
            'sharpe_ratio',
            'max_drawdown',
            'turnover',
            'cost_drag',
        ],
        'alert_thresholds': {
            'sharpe_ratio': {'min': 0.0},
            'max_drawdown': {'max': -0.20},
            'turnover': {'max': CONFIG['execution']['turnover_cap_annual'] / 252},
        },
    },
    'current_snapshot': {
        'sharpe_ratio': backtest_report['net']['sharpe'],
        'max_drawdown': backtest_report['net']['max_drawdown'],
        'turnover': json.load(open(DIRS['execution'] / 'turnover.json'))['mean'],
        'cost_drag': backtest_report['cost_impact']['return_drag'],
    },
}

# Save artifacts
safe_write_json(DIRS['governance'] / 'acceptance_draft.json', acceptance_draft)
safe_write_json(DIRS['governance'] / 'governance_pack.json', governance_pack)
safe_write_json(DIRS['monitoring'] / 'monitoring_pack.json', monitoring_pack)

# Register artifacts
add_artifact('acceptance_draft', DIRS['governance'] / 'acceptance_draft.json',
             sha256_json(acceptance_draft), 'v1.0', ['test_results', 'backtest_report'])
add_artifact('governance_pack', DIRS['governance'] / 'governance_pack.json',
             sha256_json(governance_pack), 'v1.0', ['acceptance_draft'])
add_artifact('monitoring_pack', DIRS['monitoring'] / 'monitoring_pack.json',
             sha256_json(monitoring_pack), 'v1.0', ['backtest_report'])

print("=== GOVERNANCE ARTIFACTS ===")
print(f"Acceptance status: {acceptance_draft['overall_status']}")
print(f"\nCriteria:")
for name, criteria in acceptance_criteria.items():
    status = "✓ PASS" if criteria['passed'] else "✗ FAIL"
    required = " (REQUIRED)" if criteria['required'] else " (optional)"
    print(f"  {name:20s}: {status}{required}")
    print(f"    {criteria['description']}")

print(f"\nApprovals pending:")
for role, approval in governance_pack['approvals'].items():
    print(f"  {role}: {approval['status']}")

print()

# Cell 16 — Agentic handoff demonstration (Ch.24 alignment)

"""
Agentic Handoff Demonstration

Simulates specialized agents that:
- Read artifact bundles
- Analyze results within their domain
- Produce memos with findings and recommendations
- Create handoff bundles for cross-functional review

Agents:
- ResearchAgent: Analyzes backtest, suggests improvements
- RiskAgent: Monitors limits, drawdowns, turnover
- OpsAgent: Checks data quality, runtime stats
"""

# Agent 1: Research Agent
print("Running Research Agent...")

# Read allowed artifacts
backtest_report = json.load(open(DIRS['reports'] / 'backtest_report.json'))
attribution = json.load(open(DIRS['reports'] / 'attribution.json'))
failure_modes = json.load(open(DIRS['reports'] / 'failure_modes.json'))

research_findings = {
    'performance_summary': {
        'net_sharpe': backtest_report['net']['sharpe'],
        'net_cagr': backtest_report['net']['cagr'],
        'max_drawdown': backtest_report['net']['max_drawdown'],
    },
    'observations': [
        f"Net Sharpe ratio of {backtest_report['net']['sharpe']:.2f} indicates {'positive' if backtest_report['net']['sharpe'] > 0 else 'negative'} risk-adjusted returns",
        f"Cost drag of {backtest_report['cost_impact']['return_drag']:.2%} suggests transaction costs are material",
        f"Identified {len(failure_modes['large_drawdowns'])} periods with significant drawdowns",
    ],
    'recommendations': [
        "Consider reducing turnover to minimize cost drag",
        "Investigate regime-specific performance (high vol vs low vol)",
        "Add robustness checks: out-of-sample validation, parameter sensitivity",
    ],
}

research_memo = {
    'agent': 'ResearchAgent',
    'timestamp': datetime.now().isoformat(),
    'inputs_used': [
        'backtest_report',
        'attribution',
        'failure_modes',
    ],
    'inputs_hashes': {
        'backtest_report': sha256_json(backtest_report),
        'attribution': sha256_json(attribution),
        'failure_modes': sha256_json(failure_modes),
    },
    'findings': research_findings,
    'severity': 'medium',
    'next_steps': [
        'Run parameter sweep to optimize signal weights',
        'Conduct split-sample validation',
        'Compare performance across regime periods',
    ],
}

safe_write_json(DIRS['agents'] / 'memo_research.json', research_memo)

# Simulated transcript
research_transcript = {
    'agent': 'ResearchAgent',
    'session_id': f"{RUN_ID}_research",
    'interactions': [
        {
            'step': 1,
            'action': 'load_artifacts',
            'artifacts': ['backtest_report.json', 'attribution.json'],
        },
        {
            'step': 2,
            'action': 'analyze_performance',
            'result': 'Sharpe ratio positive but cost drag significant',
        },
        {
            'step': 3,
            'action': 'generate_recommendations',
            'result': 'Suggest turnover reduction and robustness checks',
        },
    ],
}
safe_write_json(DIRS['agents'] / 'transcript_research.json', research_transcript)

print("  ✓ Research memo generated")

# Agent 2: Risk Agent
print("Running Risk Agent...")

# Read allowed artifacts
risk_events = json.load(open(DIRS['risk'] / 'risk_events.json'))
constraints_bound = json.load(open(DIRS['portfolio'] / 'constraints_bound.json'))
backtest_report = json.load(open(DIRS['reports'] / 'backtest_report.json'))

risk_findings = {
    'limit_breaches': {
        'max_leverage': constraints_bound['max_leverage_bound'],
        'leverage_limit': CONFIG['portfolio']['max_leverage'],
        'breach': constraints_bound['max_leverage_bound'] > CONFIG['portfolio']['max_leverage'],
    },
    'drawdown_events': len(risk_events['drawdown_events']),
    'circuit_breaker_triggers': len(risk_events['circuit_breaker_events']),
    'max_drawdown': backtest_report['net']['max_drawdown'],
    'observations': [
        f"Max drawdown of {backtest_report['net']['max_drawdown']:.2%} is within acceptable range",
        f"Circuit breaker triggered {len(risk_events['circuit_breaker_events'])} times",
        f"Drawdown control activated {len(risk_events['drawdown_events'])} times",
    ],
}

risk_memo = {
    'agent': 'RiskAgent',
    'timestamp': datetime.now().isoformat(),
    'inputs_used': [
        'risk_events',
        'constraints_bound',
        'backtest_report',
    ],
    'inputs_hashes': {
        'risk_events': sha256_json(risk_events),
        'constraints_bound': sha256_json(constraints_bound),
        'backtest_report': sha256_json(backtest_report),
    },
    'findings': risk_findings,
    'severity': 'low',
    'next_steps': [
        'Monitor real-time drawdown metrics in production',
        'Review circuit breaker threshold if triggers are frequent',
        'Validate volatility targeting effectiveness',
    ],
}

safe_write_json(DIRS['agents'] / 'memo_risk.json', risk_memo)
print("  ✓ Risk memo generated")

# Agent 3: Ops Agent
print("Running Ops Agent...")

# Read allowed artifacts
data_quality = json.load(open(DIRS['data'] / 'data_quality.json'))
turnover = json.load(open(DIRS['execution'] / 'turnover.json'))

ops_findings = {
    'data_quality': {
        'missing_rate': data_quality['missing_rate'],
        'acceptable': data_quality['missing_rate'] < 0.01,
    },
    'turnover': {
        'mean': turnover['mean'],
        'max': turnover['max'],
        'exceeded_cap': turnover['exceeded_cap'],
    },
    'observations': [
        f"Data missing rate of {data_quality['missing_rate']:.2%} is acceptable",
        f"Turnover exceeded cap {turnover['exceeded_cap']} times",
        f"Average turnover of {turnover['mean']:.4f} is {'sustainable' if turnover['mean'] < 0.1 else 'high'}",
    ],
}

ops_memo = {
    'agent': 'OpsAgent',
    'timestamp': datetime.now().isoformat(),
    'inputs_used': [
        'data_quality',
        'turnover',
    ],
    'inputs_hashes': {
        'data_quality': sha256_json(data_quality),
        'turnover': sha256_json(turnover),
    },
    'findings': ops_findings,
    'severity': 'low',
    'next_steps': [
        'Monitor data quality in live feeds',
        'Set up alerts for missing data',
        'Track execution quality metrics',
    ],
}

safe_write_json(DIRS['agents'] / 'memo_ops.json', ops_memo)
print("  ✓ Ops memo generated")

# Create handoff bundle
handoff_bundle = {
    'run_id': RUN_ID,
    'timestamp': datetime.now().isoformat(),
    'agents': ['ResearchAgent', 'RiskAgent', 'OpsAgent'],
    'memos': {
        'research': 'agents/memo_research.json',
        'risk': 'agents/memo_risk.json',
        'ops': 'agents/memo_ops.json',
    },
    'key_artifacts': {
        'backtest_report': 'reports/backtest_report.json',
        'test_results': 'validation/test_results.json',
        'acceptance_draft': 'governance/acceptance_draft.json',
    },
    'overall_recommendation': 'APPROVE_WITH_MONITORING',
    'rationale': 'System passes acceptance criteria, performance is positive, risks are manageable',
}

safe_write_json(DIRS['agents'] / 'handoff_bundle.json', handoff_bundle)

# Register artifacts
add_artifact('memo_research', DIRS['agents'] / 'memo_research.json',
             sha256_json(research_memo), 'v1.0', ['backtest_report'])
add_artifact('memo_risk', DIRS['agents'] / 'memo_risk.json',
             sha256_json(risk_memo), 'v1.0', ['risk_events'])
add_artifact('memo_ops', DIRS['agents'] / 'memo_ops.json',
             sha256_json(ops_memo), 'v1.0', ['data_quality'])
add_artifact('handoff_bundle', DIRS['agents'] / 'handoff_bundle.json',
             sha256_json(handoff_bundle), 'v1.0',
             ['memo_research', 'memo_risk', 'memo_ops'])

print("\n=== AGENTIC HANDOFF COMPLETE ===")
print(f"Agents run: {len(handoff_bundle['agents'])}")
print(f"Overall recommendation: {handoff_bundle['overall_recommendation']}")
print(f"Rationale: {handoff_bundle['rationale']}")
print()

# Cell 17 — Final summary report and directory index

"""
Final Summary Report

Produces:
- Complete artifact index with hashes
- Run summary with key metrics
- Confirmation of end-to-end completion
"""

# Save final artifact registry
save_artifact_registry()

# Generate artifact index
artifact_index = {
    'run_id': RUN_ID,
    'total_artifacts': len(ARTIFACT_REGISTRY),
    'artifacts': ARTIFACT_REGISTRY,
}

safe_write_json(DIRS['manifest'] / 'artifact_index.json', artifact_index)

# Generate run summary
backtest_report = json.load(open(DIRS['reports'] / 'backtest_report.json'))
acceptance_draft = json.load(open(DIRS['governance'] / 'acceptance_draft.json'))

run_summary_text = f"""
================================================================================
FOUNDATIONS OF MODERN ALGORITHMIC TRADING
Chapter 25: Capstone System - Run Summary
================================================================================

RUN METADATA
------------
Run ID: {RUN_ID}
Seed: {SEED}
Timestamp: {datetime.now().isoformat()}
Config Hash: {sha256_json(CONFIG)[:16]}...

SYSTEM CONFIGURATION
--------------------
Assets: {CONFIG['data']['num_assets']}
Bars: {CONFIG['data']['num_bars']}
Execution Delay: {CONFIG['time_semantics']['execution_delay_bars']}
Max Leverage: {CONFIG['portfolio']['max_leverage']}
Regime Enabled: {CONFIG['regime']['enabled']}

PERFORMANCE METRICS
-------------------
Gross CAGR: {backtest_report['gross']['cagr']:.2%}
Gross Sharpe: {backtest_report['gross']['sharpe']:.2f}
Gross Max DD: {backtest_report['gross']['max_drawdown']:.2%}

Net CAGR: {backtest_report['net']['cagr']:.2%}
Net Sharpe: {backtest_report['net']['sharpe']:.2f}
Net Max DD: {backtest_report['net']['max_drawdown']:.2%}

Benchmark CAGR: {backtest_report['benchmark']['cagr']:.2%}
Benchmark Sharpe: {backtest_report['benchmark']['sharpe']:.2f}

Cost Drag: {backtest_report['cost_impact']['return_drag']:.2%}

GOVERNANCE STATUS
-----------------
Acceptance: {acceptance_draft['overall_status']}
Artifacts: {len(ARTIFACT_REGISTRY)}
Tests Passed: {sum(1 for t in json.load(open(DIRS['validation'] / 'test_results.json'))['tests'] if t['passed'])}/{len(json.load(open(DIRS['validation'] / 'test_results.json'))['tests'])}

KEY ARTIFACTS
-------------
Config: manifest/config.json
Data: data/adj_prices.npy, data/returns.npy
Features: features/features.npy
Signals: signals/signals_combined.npy
Portfolio: portfolio/weights_target.npy, risk/weights_final.npy
Execution: execution/fills.npy, execution/costs.npy
Backtest: reports/backtest_report.json
Tests: validation/test_results.json
Governance: governance/acceptance_draft.json
Agents: agents/handoff_bundle.json

ARTIFACT DIRECTORY
------------------
{BASE_DIR}/
  manifest/ - Configuration and registry
  data/ - Raw and normalized data
  features/ - Engineered features
  signals/ - Baseline and combined signals
  portfolio/ - Target weights
  risk/ - Final weights with overlays
  execution/ - Orders, fills, costs
  validation/ - Test results
  monitoring/ - Monitoring pack
  governance/ - Acceptance and approvals
  reports/ - Backtest results
  agents/ - Agent memos and handoffs

================================================================================
NOTEBOOK COMPLETED SUCCESSFULLY
All stages executed, all artifacts generated, governance ready for review.
================================================================================
"""

safe_write_txt(DIRS['reports'] / 'run_summary.txt', run_summary_text)

print(run_summary_text)

# Create final visualizations
print("Generating final visualizations...")

# Plot 1: Drawdown analysis
equity = np.load(DIRS['reports'] / 'backtest_equity.npy')[:, 1]  # Net equity
running_max = np.maximum.accumulate(equity)
drawdown = (equity - running_max) / running_max

plt.figure(figsize=(12, 4))
plt.fill_between(range(len(drawdown)), drawdown, 0, alpha=0.3)
plt.plot(drawdown, linewidth=1)
plt.axhline(y=-CONFIG['risk']['drawdown_threshold'], color='r', linestyle='--',
            label=f"Threshold ({-CONFIG['risk']['drawdown_threshold']:.0%})")
plt.xlabel('Time (bars)')
plt.ylabel('Drawdown')
plt.title('Drawdown Analysis')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(DIRS['reports'] / 'drawdown_analysis.png', dpi=150)
plt.close()

# Plot 2: Turnover over time
turnover_data = json.load(open(DIRS['execution'] / 'turnover.json'))
turnover_series = np.array(turnover_data['total'])

plt.figure(figsize=(12, 4))
plt.plot(turnover_series, alpha=0.7, linewidth=1)
plt.axhline(y=CONFIG['execution']['turnover_cap_annual'] / 252, color='r',
            linestyle='--', label='Daily Cap')
plt.xlabel('Time (bars)')
plt.ylabel('Turnover')
plt.title('Turnover Over Time')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(DIRS['reports'] / 'turnover_analysis.png', dpi=150)
plt.close()

print("Visualizations saved to reports/")
print()

print("=" * 80)
print("CAPSTONE SYSTEM COMPLETE")
print("=" * 80)
print(f"\nAll artifacts saved to: {BASE_DIR}")
print(f"Total artifacts: {len(ARTIFACT_REGISTRY)}")
print(f"Acceptance status: {acceptance_draft['overall_status']}")
print(f"\nNext steps:")
print("  1. Review governance/acceptance_draft.json")
print("  2. Review agents/handoff_bundle.json")
print("  3. Obtain approvals from stakeholders")
print("  4. Deploy to production with monitoring")
print()

# Cell 18 — Optional: Real Data Adapter (yfinance) - DISABLED BY DEFAULT

"""
OPTIONAL: Real Data Adapter (yfinance)

This cell is DISABLED by default. To enable:
1. Install yfinance: !pip install yfinance
2. Set ENABLE_REAL_DATA = True below
3. Run this cell

This adapter fetches real market data and converts it to the same format
as synthetic data, allowing the entire pipeline to run with real data.

NOTE: This is optional and not required for Chapter 25 completion.
"""

ENABLE_REAL_DATA = False  # Set to True to enable

if ENABLE_REAL_DATA:
    print("=" * 80)
    print("REAL DATA ADAPTER (OPTIONAL)")
    print("=" * 80)

    try:
        import yfinance as yf
        print("yfinance imported successfully")

        # Define real tickers
        real_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'META']
        N_real = len(real_tickers)

        # Fetch data
        print(f"Fetching data for {N_real} tickers...")
        start_date = '2020-01-01'
        end_date = '2023-12-31'

        data = yf.download(real_tickers, start=start_date, end=end_date, progress=False)

        # Extract adjusted close and volume
        if N_real == 1:
            adj_close = data['Adj Close'].values.reshape(-1, 1)
            volume = data['Volume'].values.reshape(-1, 1)
        else:
            adj_close = data['Adj Close'].values
            volume = data['Volume'].values

        T_real = len(adj_close)

        print(f"Downloaded {T_real} bars for {N_real} assets")

        # Convert to same format as synthetic data
        real_raw_prices = adj_close
        real_raw_volumes = volume

        # Save to alternate directory (don't overwrite synthetic data)
        real_data_dir = BASE_DIR / 'data_real'
        real_data_dir.mkdir(exist_ok=True)

        safe_write_npy(real_data_dir / 'raw_prices.npy', real_raw_prices)
        safe_write_npy(real_data_dir / 'raw_volumes.npy', real_raw_volumes)

        real_universe_manifest = {
            'tickers': real_tickers,
            'num_assets': N_real,
            'source': 'yfinance',
            'start_date': start_date,
            'end_date': end_date,
        }
        safe_write_json(real_data_dir / 'universe_manifest.json', real_universe_manifest)

        print(f"\nReal data saved to {real_data_dir}")
        print(f"To use real data: replace CONFIG['data'] paths and re-run pipeline")
        print("\nReal data shape:")
        print(f"  Prices: {real_raw_prices.shape}")
        print(f"  Volumes: {real_raw_volumes.shape}")

    except ImportError:
        print("ERROR: yfinance not installed. Install with: !pip install yfinance")
    except Exception as e:
        print(f"ERROR: Failed to fetch real data: {e}")

else:
    print("=" * 80)
    print("REAL DATA ADAPTER - DISABLED")
    print("=" * 80)
    print("\nTo enable real data:")
    print("  1. Install yfinance: !pip install yfinance")
    print("  2. Set ENABLE_REAL_DATA = True in this cell")
    print("  3. Re-run this cell")
    print("\nThis is optional - the notebook is complete without real data.")
    print()

=== RUN METADATA ===
RUN_ID: run_42_a043bae1
SEED: 42
Base Directory: runs/run_42_a043bae1
Timestamp: 2026-01-05T13:04:47.407180

=== DIRECTORY TREE ===
manifest       : runs/run_42_a043bae1/manifest
data           : runs/run_42_a043bae1/data
features       : runs/run_42_a043bae1/features
signals        : runs/run_42_a043bae1/signals
portfolio      : runs/run_42_a043bae1/portfolio
risk           : runs/run_42_a043bae1/risk
execution      : runs/run_42_a043bae1/execution
validation     : runs/run_42_a043bae1/validation
monitoring     : runs/run_42_a043bae1/monitoring
governance     : runs/run_42_a043bae1/governance
reports        : runs/run_42_a043bae1/reports
agents         : runs/run_42_a043bae1/agents

=== CONFIGURATION ===
Config saved to: runs/run_42_a043bae1/manifest/config.json
Config hash: c1b69e1f4eeb3c58...

Key parameters:
  Assets: 10
  Bars: 1000
  Execution delay: 1
  Max leverage: 1.0
  Regime enabled: True

=== UTILITIES LOADED ===
Available functions:
  - Hashing: sha25

##3.CONCLUSIONS

**Concluding Section — Alex, Ten Days to Committee**

**Day 1 Reality Check: What “Strategy” Means Depends on Who’s Listening**  
I am Alex. I joined Trading Operations on a Monday morning, and by lunchtime I learned two facts that no textbook ever puts on the cover. First, the word “strategy” means radically different things depending on who is speaking. Research hears **alpha hypothesis**. Risk hears **loss pathways**. Compliance hears **process exposure**. Operations hears **breakage probability**. Technology hears **interfaces and failure modes**. The committee hears **should we approve this and live with it**. Second, nobody cares how elegant my idea is if I cannot prove three things: **accountability**, **traceability**, and **production survivability**. I am not here to be clever in private. I am here to help the firm make a decision it can defend.

**The Ten-Day Constraint: Short Enough to Hurt, Long Enough to Reveal Bureaucracy**  
I have ten days to present a strategy to the committee. That timeline is short enough to be uncomfortable, and long enough to reveal the bureaucracy that keeps a trading organization alive. The bureaucracy is not paperwork for its own sake. It is the machinery that turns **a model on a laptop** into **a system the firm is willing to run**. If I fight it, I lose. If I learn it, it becomes my leverage. The capstone notebook I inherited is my map. It does not just compute performance; it demonstrates a full lifecycle: **research → promotion → production**, with artifacts that make each stage defensible.

**What I Must Deliver: Four Outputs, Not One Chart**  
If I walk into the committee room with a single equity curve, I will lose credibility within minutes. I need four deliverables aligned to lifecycle phases: **(a) contracts**, **(b) research and gates**, **(c) promotion and approvals**, and **(d) production-style simulation**. Each deliverable has boundaries that tell me what I own, what I influence, and what must be handed off. This is also the first place where I must understand **separation of concerns**: without it, I will either try to own everything and burn out, or own nothing and become irrelevant.

---

**A. Contracts First: Data, Time, and Accountability Before Performance**

**Why Contracts Come Before Backtests**  
On day one, the temptation is to run the backtest. That is what a lone researcher would do. In Trading Operations, that is a trap. The committee’s first question is rarely “what is the Sharpe?” It is almost always: **what data is this based on**, **do we have the right to use it**, and **can we reproduce what you are showing us**. If I cannot answer those, performance is irrelevant. So my first deliverable is not a performance chart. It is a **data contract pack**.

**What My Data Contract Pack Contains**  
I treat the dataset the way the capstone notebook treats it: not as a blob of arrays, but as a governed object with a **manifest**, **quality checks**, and a **fingerprint**. My contract pack includes: a clear universe definition (what instruments exist and under what rules), explicit time semantics (decision time, execution time, and delay), corporate action handling (who provides adjustment factors and how gaps are treated), missing data policy (forward fill, drop, or fail closed), data quality thresholds (what triggers an alert), and dataset fingerprinting (how we uniquely identify “this run used this dataset”). The practical message is: **the firm cannot audit what it cannot identify**.

**Where My Responsibility Begins and Ends in Contracts**  
I do not own the data feed. I do not own legal licensing. I do not own vendor relationships. What I own is the **specification of how the strategy uses data** and the **evidence that I used it consistently**. I work with Data Engineering and Compliance using their existing templates. I do not invent a new bureaucracy; I translate the capstone artifacts into the firm’s language. This is bureaucracy at its best: **shared memory** that prevents future arguments.

---

**B. Research Under Gates: Time Discipline and Evidence, Not “Pretty Results”**

**Research in a Trading Org Is Not Exploration Without Rules**  
With contracts drafted, I move to research. But research here is not “run experiments until something looks good.” It is “produce a candidate system that survives gates.” The gates exist to prevent the most dangerous failure modes: hidden lookahead, nondeterminism, and frictionless fantasies. The capstone notebook’s structure forces me to keep those under control.

**Locking the Run: Determinism and a Single Source of Truth**  
I begin by locking configuration as a **single source of truth** and storing its hash. I treat that as a commitment: if I change the config, I changed the system, and I must log the change. This is not pedantry. It prevents “moving target” presentations. The committee’s trust grows when the run identity is stable.

**Causality in Features: Evidence Beats Assurances**  
Feature engineering is where many strategies die, not because the idea is wrong, but because the implementation quietly leaks future information. The capstone’s lagging rules and causality checks are my defense. In a committee room, “we are time-aware” is a claim. Leakage tests, monotonic time checks, and explicit lags are **evidence**. That evidence is what turns a narrative into something reviewable.

**Signals Are Not Portfolios: Modular Thinking Prevents Confusion**  
Next I generate signals as modular components: trend, mean reversion, and volatility inputs. I keep the boundary clean: **features measure**, **signals decide**, **portfolios allocate**. This separation is not cosmetic; it makes it possible to diagnose failures. If the backtest struggles, I can ask: is it the data, the features, the signals, the constraints, the risk overlays, or execution costs? If everything is mixed together, the only answer becomes “we’re not sure.”

**Constraints Are Part of the Model, Not a Footnote**  
Portfolio construction with explicit constraints is where research becomes real. Constraints are not optional. The firm must be able to say: “this strategy respects leverage limits, position limits, and neutrality rules at all times.” The solver trace is especially valuable because it shows how often constraints bind. If we are clipping constantly, we are effectively running a different strategy than the signal suggests. That is not a failure; it is a discovery that informs tuning.

**Risk Overlays as a Separate Control Layer**  
Risk overlays are not “more alpha.” They are **survival controls**: volatility targeting, drawdown control, and circuit breakers. Crucially, they are logged. In production, the question is not only “what happened,” but “why did you reduce exposure here?” Logged risk events provide operational explanations. This is one of the most important committee trust signals: the system can be explained **after the fact**.

**Passing the Gates: Tests Replace Confidence with Proof**  
Now I pass through validation gates: time monotonicity, no-lookahead boundary checks, reproducibility checks via hashes, cost sanity, turnover cap enforcement, and fail-closed behavior. If a test fails, I do not debate it; I fix the system. In a serious organization, arguing with tests is arguing against the firm’s ability to defend itself. That is not a battle worth fighting.

---

**C. Promotion: Turning a Model into a Decision the Firm Can Sign**

**Promotion Is a Formal Translation: Research Outputs Into Governance Artifacts**  
Even a strong backtest is not an approval. Promotion is the phase where the system becomes a candidate the firm can endorse. This is where governance becomes explicit. The committee is not only judging performance; it is judging **process maturity** and **accountability**. The capstone’s promotion artifacts give me the structure: acceptance criteria, a governance pack, and a monitoring contract.

**Acceptance Criteria: What Must Be True Before We Say “Yes”**  
I define acceptance criteria in the firm’s language: causality must pass, reproducibility must pass, cost sanity must pass, and performance must be plausible. I present these criteria as a readable checklist with evidence pointers: where the test results are stored, where the backtest report is stored, and where the config hash lives. The committee does not want improvisation. It wants a decision framework.

**Governance Pack: Approvals Are Part of the System, Not an Afterthought**  
I include approvals placeholders: research lead, risk officer, compliance. This is not ceremonial. It makes ownership visible and review structured. Each stakeholder knows what they are approving and where the evidence is. I also document the audit trail paths: artifact registry, test results, acceptance draft. This reduces the friction of “chasing proof,” which is often what kills promising strategies in governance pipelines.

**Monitoring Pack: Proving I Understand What Happens After Approval**  
Monitoring is where production begins in spirit. I define which metrics must be tracked and what thresholds trigger alerts: drawdown, turnover, cost drag, and risk-adjusted performance. If I cannot propose monitoring, I do not understand that trading is an operational process. Monitoring is not paranoia; it is how the firm stays in control when reality diverges from backtests.

---

**D. Production: Simulating Real Life Before Real Money Exists**

**Production Is Not a Place — It Is a Discipline**  
Production means: the system runs on schedule, uses reliable feeds, generates decisions at defined times, executes with realistic constraints, logs everything, and can be explained after the fact. In ten days, I cannot deploy, but I can run a production-style simulation that demonstrates readiness. That is what convinces the committee I am not selling a frictionless fantasy.

**Execution Reality: Delays, Costs, Turnover Caps, and Holdings Accounting**  
I simulate execution delay so decisions do not fill instantly. I apply spread, fees, and impact at the time of execution. I compute turnover and enforce caps. I track holdings explicitly. This is where many “great” strategies collapse into mediocrity, and that is precisely why I show it. If the strategy survives friction, it earns credibility. If it does not, it is not a candidate yet.

**Kill Switch and Drawdown Controls: Safety Must Be Demonstrable**  
I demonstrate emergency behavior: circuit breakers that go flat after severe losses, and drawdown controls that reduce exposure when the equity curve degrades. Most importantly, these events are logged. In production, a control that is not logged is indistinguishable from an accident. Logging transforms controls into accountable policy.

**Operator Experience: What Ops Needs When Things Go Wrong**  
Operations people care about what happens when the world is messy: missing data, strange volumes, failed writes, unusual costs. The artifact directory structure is not a convenience; it is the post-mortem tool. I ensure there is a run summary that tells an operator what happened without requiring my presence. That is what “accountable” looks like: the system remains intelligible when I am not in the room.

**Cross-Functional Handoff: Demonstrating Team-Based Decision Flow**  
I use structured memos to simulate cross-functional review: research memo, risk memo, ops memo. Each memo references the artifacts it used and includes hashes so the memo is traceable to a run. Then I bundle them into a handoff pack that the committee can forward. The committee does not approve loners; it approves systems with **owners** and **handoffs**.

---

**My Role Boundary: Where My Concerns Begin and End**

**What I Own**  
My concerns begin with strategy logic, decision timing, and evidence. I own input requirements, data policies, and missingness handling. I own causality in features and signals and the proof that I enforced it. I own portfolio constraints implementation, risk overlays triggers and logs, execution assumptions and timing correctness, and the gross-vs-net evaluation evidence. I own validation artifacts, reproducibility checks, acceptance criteria drafts, monitoring contracts, and handoff documents.

**What I Do Not Own (But Must Understand)**  
I do not own the market data feed uptime, licensing decisions, infrastructure deployment pipeline, or the firm’s final thresholds for capital allocation. But I must be literate enough to align to them and communicate effectively with their owners. That is what it means to be part of a team without dissolving responsibility.

---

**How I Present to the Committee: Start With Lifecycle Discipline, Not With Performance**

**The Opening Frame: Trust the Evidence Pack, Not My Confidence**  
When I present, I do not open with a chart. I open with lifecycle maturity: single source of truth config, deterministic run ID, governed data artifacts, causal features, modular signals, optional regime routing with strict timing, portfolio constraints, risk overlays, execution cost realism, timing-correct backtesting, an end-to-end test suite, acceptance criteria, monitoring contract, and cross-functional handoff memos.

Then I map this to committee concerns: Research sees the hypothesis and results, Risk sees limits and controls, Ops sees run packaging and failure behavior, Compliance sees audit trail and contracts, Tech sees interfaces and module boundaries.

Finally, I say the most important sentence of the meeting: **I am not asking you to trust me. I am asking you to trust the evidence pack.** People leave. Evidence remains. That is how the firm protects itself.

---

**The Ten-Day Lesson: The Capstone Is Not the End of Learning — It Is the Start of Operating Professionally**  
Ten days is not enough time to perfect a strategy. It is enough time to prove whether I understand how trading organizations work. I learned that the system is larger than the model. I learned that bureaucracy is shared memory, not an enemy. I learned that gates prevent self-deception. I learned that production is clarity: clear timing, clear interfaces, clear controls, clear logs, clear owners. And I learned that separation of concerns is not a design preference; it is how teams collaborate without chaos.

When I walk into the committee room, I do not walk in alone. I represent a chain: data, research, risk, operations, compliance, technology. My job is to show that I can navigate that chain, respect it, and strengthen it. If I can do that, the committee can approve the strategy not as a gamble, but as a governed process that can be monitored, audited, improved, and stopped when necessary.
